[DIY Disease Tracking Dashboard Kit](https://github.com/weavermech/coda) (C) Stephen Hanson 2024 ([stephenhansonj@gmail.com](stephenhansonj@gmail.com))  
Based on UK Government [data](https://ukhsa-dashboard.data.gov.uk/) published by the [UK Health Security Agency](https://www.gov.uk/government/organisations/uk-health-security-agency) and on the [DIY Disease Tracking Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) by Fabrizio Smeraldi. Released under the [GNU GPLv3.0 or later](https://www.gnu.org/licenses/).


In [18]:
from anytree import Node, RenderTree
from anytree.importer import JsonImporter, DictImporter
import datetime
from IPython.display import clear_output, Markdown
import ipywidgets as wdg
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import pickle
import requests
import time

import warnings
warnings.filterwarnings('ignore', category=FutureWarning) # avoiding pandas warning

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100
global regions 
regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
global filters
filters={"stratum" : None, # Smallest subgroup a metric can be broken down into e.g. ethnicity, testing pillar
    "age": None, # Smallest subgroup a metric can be broken down into e.g. 15_44 for the age group of 15-44 years
    "sex": None, #  Patient gender e.g. 'm' for Male, 'f' for Female or 'all' for all genders
    "year": None,#2022, #  Epi year of the metrics value (important for annual metrics) e.g. 2020
    "month": None, # Epi month of the metric value (important for monthly metrics) e.g. 12
    "epiweek" :None, # Epi week of the metric value (important for weekly metrics) e.g. 30
    "date" : None, # The date which this metric value was recorded in the format YYYY-MM-DD e.g. 2020-07-20
    "in_reporting_delay_period": None # Boolean indicating whether the data point is considered to be subject to retrospective updates
}

In [ ]:
# loading data for initial graphs
with open("mean_occBed_region.json", "rt") as INFILE:
    mean_occBed_region=json.load(INFILE)

with open("leaf_first.json", "rt") as INFILE:
    leaf_data =json.load(INFILE)

In [ ]:
def wrangle_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    # editied to include the geography
    data={}
    for dataset in rawdata:
        for entry in dataset:
            date=entry['date']
            value=entry['metric_value']
            geography=entry['geography']
            if date not in data:
                data[date]={}
            data[date][geography]=value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    mean_occBed_region_df=pd.DataFrame(index=index, columns=regions)

    for date, values in data.items():
        for region, value in values.items():
            mean_occBed_region_df.at[parse_date(date), region] = value
    # fill in any remaining "holes" due to missing dates
    mean_occBed_region_df.fillna(0.0, inplace=True)

    return mean_occBed_region_df


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in this cell as below:
df1=wrangle_data(mean_occBed_region) # df is the dataframe for plotting

2020-08-07 00:00:00  to  2024-10-31 00:00:00


In [ ]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the UKHSA API. Return data as a like-for-like replacement for the "canned" data loaded from the JSON file. """
    
    class APIwrapper:
        # class variables shared among all instances
        _access_point="https://api.ukhsa-dashboard.data.gov.uk/v2"       
        _last_access=0.0 # time of last api access
        root = Node("root")
        
        def __init__(self, theme, sub_theme, topic, geography_type, geography, metric):
            """ Init the APIwrapper object, constructing the endpoint from the structure
            parameters """
            # build the path with all the required structure parameters. You do not need to edit this line,
            # parameters will be replaced by the actual values when you instantiate an object of the class!
            url_path=(f"/themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/" +
                    f"{geography_type}/geographies/{geography}/metrics/{metric}")
            # our starting API endpoint
            self._start_url=APIwrapper._access_point+url_path
            self._filters=None
            self._page_size=-1
            # will contain the number of items
            self.count=None



        def get_page(self, filters={}, page_size=5):
            """ Access the API and download the next page of data. Sets the count
            attribute to the total number of items available for this query. Changing
            filters or page_size will cause get_page to restart from page 1. Rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365); use the default value 
            for debugging your structure and filters. """
            # Check page size is within range
            if page_size>365:
                raise ValueError("Max supported page size is 365")
            # restart from first page if page or filters have changed
            if filters!=self._filters or page_size!=self._page_size:
                self._filters=filters
                self._page_size=page_size
                self._next_url=self._start_url
            # signal the end of data condition
            if self._next_url==None: 
                return [] # we already fetched the last page
            # simple rate limiting to avoid bans
            curr_time=time.time() # Unix time: number of seconds since the Epoch
            deltat=curr_time-APIwrapper._last_access
            if deltat<0.33: # max 3 requests/second
                time.sleep(0.33-deltat)
            APIwrapper._last_access=curr_time
            # build parameter dictionary by removing all the None
            # values from filters and adding page_size
            parameters={x: y for x, y in filters.items() if y!=None}
            parameters['page_size']=page_size
            # the page parameter is already included in _next_url.
            # This is the API access. Response is a dictionary with various keys.
            # the .json() method decodes the response into Python object (dictionaries,
            # lists; 'null' values are translated as None).
            response = requests.get(self._next_url, params=parameters).json()
            # update url so we'll fetch the next page
            self._next_url=response['next']
            self.count=response['count']
            # data are in the nested 'results' list
            return response['results'] 

        def get_all_pages(self, filters={}, page_size=365):
            """ Access the API and download all available data pages of data. Sets the count
            attribute to the total number of items available for this query. API access rate
            limited to three request per second. The page_size parameter sets the number
            of data points in one response page (maximum 365), and controls the trade-off
            between time to load a page and number of pages; the default should work well 
            in most cases. The number of items returned should in any case be equal to 
            the count attribute. """
            data=[] # build up all data here
            while True:
                # use get_page to do the job, including the pacing
                next_page=self.get_page(filters, page_size)
                if next_page==[]:
                    break # we are done
                data.extend(next_page)
            return data


        # /themes/{theme}/sub_themes/{sub_theme}/topics/{topic}/geography_types/{geography_type}/geographies/{geography}/metrics/{metric}
        def add_branch_to_node(self, upper_node=None, link=None):
        
            # to kick off the tree from scratch
            # had some problems with using defualt values in the function so dealing with it here
            if upper_node is None:
                upper_node = APIwrapper.root
            if link is None:
                link = APIwrapper._access_point + '/themes/'
                
            importer = DictImporter()
            curr_time = time.time()  # Unix time: number of seconds since the Epoch
            deltat = curr_time - APIwrapper._last_access
            if deltat < 0.33:  # max 3 requests/second
                time.sleep(0.33 - deltat)
            APIwrapper._last_access = curr_time

            branches = requests.get(link).json()
            if branches:                                                # make sure the branch isn't empty                          
                for branch in branches:
                    new_node = importer.import_(branch)                 # create a new node from the branch using the dictionary importer from anytree
                    new_node.parent = upper_node
                    new_node.name = branch['name']                      # Use the value of the "name" dictionary key

                    curr_time = time.time()                             # Unix time: number of seconds since the Epoch
                    deltat = curr_time - APIwrapper._last_access
                    if deltat < 0.33:                                   # max 3 requests/second
                        time.sleep(0.33 - deltat)
                    APIwrapper._last_access = curr_time
                    next_level_link = branch['link']                    # get the link to the next section in the api

                    if "/metrics/" in next_level_link:                  # hacky way to get to find if we are at the bottom of the tree (metric data is next)
                        continue                                        # continue to next branch

                    options_link = requests.get(next_level_link).json() # get the link that returns the options for the next branch
                    options_link = list(options_link[0].values())[0]
                    
                    self.add_branch_to_node(new_node, options_link)     # recursively add the next branch to the tree

            return
    
    # This Next Line Builds The Tree  --  !!It Takes Over 50min To Run on Local Machine So It Is Commented Out And Runs From File!!
    # api.add_branch_to_node()
    
    # Load the tree from a pickle file instead
    global root
    with open('full_tree_named.pkl', 'rb') as f:
        root = pickle.load(f)

    structure={"theme": "infectious_disease", 
        "sub_theme": "respiratory",
        "topic": "COVID-19", "metric": "COVID-19_healthcare_occupiedBedsRollingMean", "geography_type": "NHS Region"} # leave out geography, will be dynamic
    
    # build the raw data
    mean_occBed_region = []
    for idx, r in enumerate(regions):
        structure["geography"] = r
        api=APIwrapper(**structure)
        mean_occBed_region.append(api.get_all_pages(filters))                    
        print(f"Data points expected: {api.count}")
        print(f"Data points retrieved: {len(mean_occBed_region[idx])}")

    # is a one-off save to create the file. Keep for future reference though
    # with open("mean_occBed_region.json", "wt") as OUTF:
    #     json.dump(mean_occBed_region, OUTF)
        
    return mean_occBed_region

In [ ]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    apidata=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df1, df2
    global regions 
    regions = ["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"]
    df1=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # The function needs to be adapted to your graph; you can omit this call
    # in the first instance
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. If you are 
    # implementing error handling, you can use icons "unlink" or "times" and 
    # change the button text to "Unavailable" when the api call fails.
    apibutton.icon="check"
    # apibutton.disabled=True

    
apibutton=wdg.Button(
    description='Engage', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Make it so",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
apibutton.on_click(api_button_callback) # the name of your function inside these brackets

# line below commented to display later
# display(apibutton)

# run all cells before clicking on this button

Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
Data points expected: 1547
Data points retrieved: 1547
2020-08-07 00:00:00  to  2024-10-31 00:00:00


## First Interactive Graph - Mean Beds Occupied by Region and Time Period 

This graph displays regional data for the mean number of beds occupied. The data for 7 English regions from Aug 2020 to Oct 2024 are availible.  
  
- Use the multi-select list to select regions (hold shift or ctrl and click to select multiple). The graph dymamically updates as the region is changed.  
- Use the date sliders to specify the start and end dates for the plot. The graph updates as the start and end dates are changed, and the y-axis scales appropriately for the data.

In [ ]:
def update_plot(selected_regions, date_range):
    fig, ax = plt.subplots(figsize=(12, 6))
    selected_regions = list(selected_regions)                   # make a list of the regions to filter by
    start_date, end_date = date_range                           # pull the start and end date out of the tuple              
    df1a = df1.loc[start_date:end_date, selected_regions].copy()               
    df1a.plot(ax=ax)
    ax.set_title("Mean Beds Occupied by Region")
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x', which='minor', rotation=65)
    ax.tick_params(axis='x', which='major', rotation=65)
    ax.set_xlabel('Date')
    ax.set_ylabel('Metric Value')

    # started by filtering the visible plot, but since df is a reasonable size decided to filter data instead
    # keeping for future reference
    # ax.set_xlim([start_date, end_date])   
                        
    # since filtering date by the data, can scale the y-axis based on the date range
    ax.set_ylim([df1a.min().min(), df1a.max().max()+50])
    
    plt.show()
  
  
reg = wdg.SelectMultiple(
    options=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    value=["North East and Yorkshire", "North West", "Midlands", "South West", "East of England", "London", "South East"],
    description='Regions',
    disabled=False
)
reg.layout.height = '120px'



dates = pd.date_range(start='2020-08-01', end='2024-10-31', freq='MS')
options = [(i.strftime('%b %Y'), i) for i in dates]
dates_slider = wdg.SelectionRangeSlider(
    options=options,
    index=(0, 50),
    description='Period',
    disabled=False,
    layout=wdg.Layout(width='40%')
)

# forcably prevent the start date equalling the end date. Might be improved.
def validate_date_range(change):
    start, end = change['new']
    if start.year == end.year and start.month == end.month:
        if start == dates[0]:
            end = dates[1]
        else:
            start = dates[dates.get_loc(end) - 1]
        yr.value = (start, end)

dates_slider.observe(validate_date_range, names='value')



def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=reg.value
    #print(current)
    if current==reg.options[0]:
        other=reg.options[1]
        #print("was reg.options[0]",other)
    else:
        other=reg.options[0]
        #print("was reg.options[1]",other)
    reg.value=(other,) # forces the redraw
    reg.value=current # now we can change it back
    
controls0=wdg.HBox([apibutton, reg, dates_slider]) 

# connect the plotting function and the widget    
graph0=wdg.interactive_output(update_plot, {'selected_regions': reg, 'date_range': dates_slider})

# actually displays the graph
display(controls0, graph0)

Output()

# Interactive Graph 2 - Access to the Full API Data

I'd heard of tree datastuctures a few years ago and decided to map the whole API to a tree structure. this took nearly an hour on my machine so it is written to a pickle file and loaded from there. The mapping code is included in the function `add_branch_to_node`. Each node has a dictinoary containing the api link and the name, and the function explores all branches down to the metrics data. The code checks when a dropdown is changed and updates the following dropdowns so you can't have parent showing that doesn't belong to a child that is shown in another dropdown.  
I wanted to try pulling the data from the link in the Metrics dictionary, rather than building a link with the structure, so retrieved the pages outside the APIwrapper class.


In [8]:
# load the pickle of the tree (if not loaded from file takes nearly an hour to generate)
with open('full_tree_named.pkl', 'rb') as f:
    root = pickle.load(f)




# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def getMetric_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    leaf_data = requests.get(metric_link, params={ 'page_size': 1800, 'page': 1}).json()
    
    # One-off to save json data to file
    # with open("leaf_first.json", "wt") as OUTF:
    #     json.dump(leaf_data, OUTF)
    
    # wrangle the data and overwrite the dataframe for plotting
    global df2
    df2=wrangle_leaf_data(leaf_data)
    
    refresh_graph()

    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. If you are 
    # implementing error handling, you can use icons "unlink" or "times" and 
    # change the button text to "Unavailable" when the api call fails.
    getMetric_button.icon="check"
    # apibutton.disabled=True

    
getMetric_button=wdg.Button(
    description='Engage', # you may want to change this...
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Make it so",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='exclamation-triangle'
)

# remember to register your button callback function with the button
getMetric_button.on_click(getMetric_button_callback) # the name of your function inside these brackets




def wrangle_leaf_data(rawdata):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    
    def parse_date(datestring):
        """ Convert a date string into a pandas datetime object """
        return pd.to_datetime(datestring, format="%Y-%m-%d")

    data={}
    for entry in rawdata['results']:
        date=entry['date']
        value=entry['metric_value']
        if date not in data:
            data[date] = value

    dates=list(data.keys())
    dates.sort()

    startdate=parse_date(dates[0])
    enddate=parse_date(dates[-1])

    index=pd.date_range(startdate, enddate, freq='D')
    leaf_metric_df = pd.DataFrame(index=index, columns=['Metric'])

    for date, value in data.items(): # each entry is a dictionary with cases, admissions and deaths
        pd_date=parse_date(date) # convert to Pandas format
        # do not assume all values are there for every date - if a value is not available, insert a 0.0
        # this is the way you access a specific location in the dataframe - use .loc
        # and put index,column in a single set of [ ]
        leaf_metric_df.loc[date, 'Metric'] = value
    leaf_metric_df.fillna(0.0, inplace=True)

    return leaf_metric_df



def plot_metric(change, title):                                        # revieves a change parameter from the widget to refresh, but not used
    fig, ax = plt.subplots(figsize=(14, 6))             
    df2.plot(ax=ax)
    ax.set_title(title)
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.xaxis.set_minor_formatter(mdates.DateFormatter('%b'))
    ax.tick_params(axis='x', which='minor', rotation=80)
    ax.tick_params(axis='x', which='major', rotation=80)
    ax.set_xlabel('Date')
    ax.set_ylabel('Metric Value')

    plt.show()

    
# helper dropdown to change without impact, and update graph. Not displayed in control1
helper=wdg.Dropdown(
    options=['One', 'Two'],
    value='One',
    description='helper',
    disabled=False,
)


def refresh_graph():
    """Change the value of the widget in order to force a redraw of the graph
    since we pull from the API every time with this graph.
    Don't want the useful dropdowns to change as they can trigger a reset and also
    not all dropdowns have >1 choice."""
    current=helper.value
    if current==helper.options[0]:
        other=helper.options[1]
    else:
        other=helper.options[0]
    helper.value=other # forces the redraw
    helper.value=current # now we can change it back



    

# function to create a dropdown widgets, rather than one by one
def create_dropdown(description, options):
    return wdg.Dropdown(
        options=options,
        value=options[0][1] if options else None,
        description=description,
        disabled=False,
        style={'description_width': '90px'},
        layout=wdg.Layout(width='30%')
    )


# problem with changing earlier dropdowns messing up later dropdown values, e.g. changing sub theme after topics were chosen
# so need to update them all
def update_options(dropdown, options):
    dropdown.options = options
    if options:
        dropdown.value = options[0][1]
    else:
        dropdown.value = None

# update each dropdown
def update_subthemes(*args):
    if theme_dd.value is not None:  # it has a value that might not work with a previous selection (not a problem for theme at the moment but future proofing)
        update_options(subTheme_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children)])
        update_topics()

def update_topics(*args):
    if subTheme_dd.value is not None:
        update_options(topic_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children)])
        update_geotypes()

def update_geotypes(*args):
    if topic_dd.value is not None:
        update_options(geoType_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children)])
        update_geographies()

def update_geographies(*args):
    if geoType_dd.value is not None:
        update_options(geographies_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children)])
        update_metrics()

def update_metrics(*args):
    if geographies_dd.value is not None:
        update_options(metrics_dd, [(child.name, idx) for idx, child in enumerate(root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children[geographies_dd.value].children)])

# create the dropdown widgets
theme_dd = create_dropdown('1. Theme:', [(child.name, idx) for idx, child in enumerate(root.children)])
subTheme_dd = create_dropdown('2. Sub Theme:', [])
topic_dd = create_dropdown('3. Topic:', [])
geoType_dd = create_dropdown('4. Geo Type:', [])
geographies_dd = create_dropdown('5. Geographies:', [])
metrics_dd = create_dropdown('6. Metrics:', [])



def update_metric_link(*args):
    global metric_link
    metric_link = root.children[theme_dd.value].children[subTheme_dd.value].children[topic_dd.value].children[geoType_dd.value].children[geographies_dd.value].children[metrics_dd.value].link


# observe the dropdowns so downstream resets when a value changes
theme_dd.observe(update_subthemes, names='value')
subTheme_dd.observe(update_topics, names='value')
topic_dd.observe(update_geotypes, names='value')
geoType_dd.observe(update_geographies, names='value')
geographies_dd.observe(update_metrics, names='value')

# Observe all dropdowns to update the metric link
theme_dd.observe(update_metric_link, names='value')
subTheme_dd.observe(update_metric_link, names='value')
topic_dd.observe(update_metric_link, names='value')
geoType_dd.observe(update_metric_link, names='value')
geographies_dd.observe(update_metric_link, names='value')
metrics_dd.observe(update_metric_link, names='value')

update_subthemes(None)

controls1 = wdg.VBox([
    wdg.HBox([theme_dd, subTheme_dd, topic_dd], layout=wdg.Layout(justify_content='space-between', width='100%')),
    wdg.HBox([geoType_dd, geographies_dd, metrics_dd], layout=wdg.Layout(justify_content='space-between', width='100%')), 
    getMetric_button
])

# plot first graph from file data
df2=wrangle_leaf_data(leaf_data) # df is the dataframe for plotting

# connect the plotting function and the widget    
graph1=wdg.interactive_output(plot_metric, {'change' : helper, 'title': wdg.fixed(metrics_dd.label)})   # get the label to send to the plotting function as a title

# actually displays the graph
display(controls1, graph1)




Output()

In [ ]:
# running from file so commented out. Kept for future reference.
 
# with open('tree_structure.txt', 'w') as outfile:
#     for pre, fill, node in RenderTree(root):
#         outfile.write("%s%s\n" % (pre, node.name))

In [19]:
with open('tree_structure.txt', 'r') as file:
    content = file.read()

display(Markdown(f"```\n{content}\n```"))

```
root
└── infectious_disease
    ├── bloodstream_infection
    │   ├── E-coli
    │   │   ├── Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and the Isles of Scilly Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Devon Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East London Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North West London Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South East London Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South West London Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-on-Trent Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex Integrated Care Board
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire Integrated Care Board
    │   │   │       └── e-coli_cases_countsByOnsetType
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── e-coli_cases_countsByOnsetType
    │   │   ├── NHS Region
    │   │   │   ├── East of England
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── London
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Midlands
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North East and Yorkshire
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North West
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── South East
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   └── South West
    │   │   │       └── e-coli_cases_countsByOnsetType
    │   │   ├── NHS Trust
    │   │   │   ├── Airedale
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── AIREDALE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Alder Hey Children's
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Ashford & St Peter's Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Barking, Havering & Redbridge University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Barnsley Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BARNSLEY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Barts Health
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BARTS HEALTH NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Birmingham Women's and Children's
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Blackpool Teaching Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Bolton NHS
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BOLTON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Bradford Teaching Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Buckinghamshire Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── BUCKINGHAMSHIRE HEALTHCARE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── CALDERDALE AND HUDDERSFIELD NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Calderdale & Huddersfield
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Cambridge University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── CHELSEA AND WESTMINSTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Chelsea & Westminster Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Chesterfield Royal Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── CHESTERFIELD ROYAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Countess of Chester Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── COUNTESS OF CHESTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── COUNTY DURHAM AND DARLINGTON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── County Durham & Darlington
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Croydon Health Services
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── CROYDON HEALTH SERVICES NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── DARTFORD AND GRAVESHAM NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Dartford & Gravesham
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── DONCASTER AND BASSETLAW TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Doncaster & Bassetlaw Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Dorset County Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── DORSET COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EAST AND NORTH HERTFORDSHIRE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── East Cheshire
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EAST CHESHIRE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── East Kent Hospitals University
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── East Lancashire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EAST LANCASHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── East & North Hertfordshire
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── East Suffolk and North Essex
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EAST SUFFOLK AND NORTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── East Sussex Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EAST SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── EPSOM AND ST HELIER UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Epsom & St Helier University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Frimley Health
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── FRIMLEY HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Gateshead Health
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── GATESHEAD HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── George Eliot Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── GEORGE ELIOT HOSPITAL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Gloucestershire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Great Ormond Street Hospital for Children
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Great Western Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── GUY'S AND ST THOMAS' NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Guy's & St. Thomas'
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Hampshire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── HARROGATE AND DISTRICT NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Harrogate & District
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── HOMERTON HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Homerton University Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Hull University Teaching Hospitals NHS Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── HULL UNIVERSITY TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Imperial College Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── IMPERIAL COLLEGE HEALTHCARE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Isle of Wight
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ISLE OF WIGHT NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── James Paget University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── JAMES PAGET UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Kettering General Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── KETTERING GENERAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── King's College Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── KING'S COLLEGE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Kingston Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── KINGSTON HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Lancashire Teaching Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LANCASHIRE TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Leeds Teaching Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LEEDS TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LEWISHAM AND GREENWICH NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Lewisham & Greenwich
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL HEART AND CHEST HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Liverpool Heart & Chest Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Liverpool University Hospitals NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Liverpool Women's
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── London North West University Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── LONDON NORTH WEST UNIVERSITY HEALTHCARE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MAIDSTONE AND TUNBRIDGE WELLS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Maidstone & Tunbridge Wells
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Manchester University
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Medway
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MEDWAY NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MERSEY AND WEST LANCASHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Mid and South Essex NHS Foundation
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MID AND SOUTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Mid Cheshire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Mid Yorkshire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MID YORKSHIRE TEACHING NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Milton Keynes University Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MILTON KEYNES UNIVERSITY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Moorfields Eye Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── MOORFIELDS EYE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORFOLK AND NORWICH UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Norfolk & Norwich University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Northampton General Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTHAMPTON GENERAL HOSPITAL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North Bristol
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTH BRISTOL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTH CUMBRIA INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Northern Care Alliance NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTHERN LINCOLNSHIRE AND GOOLE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Northern Lincolnshire & Goole
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North Middlesex University Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTH MIDDLESEX UNIVERSITY HOSPITAL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTH TEES AND HARTLEPOOL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North Tees & Hartlepool
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Northumbria Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── North West Anglia
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NORTH WEST ANGLIA NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Nottingham University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Oxford University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── OXFORD UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Portsmouth Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── PORTSMOUTH HOSPITALS UNIVERSITY NATIONAL HEALTH SERVICE TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Queen Victoria Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── QUEEN VICTORIA HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal Berkshire
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL BERKSHIRE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal Cornwall Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL CORNWALL HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal Free London
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL FREE LONDON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal National Orthopaedic Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL NATIONAL ORTHOPAEDIC HOSPITAL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal Papworth Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal Surrey County Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Royal United Hospitals Bath
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ROYAL UNITED HOSPITALS BATH NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Salisbury
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SALISBURY NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Sandwell & West Birmingham Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Sheffield Children's
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Sheffield Teaching Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Sherwood Forest Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SHERWOOD FOREST HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Shrewsbury & Telford Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Somerset NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SOMERSET NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── South Tees Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SOUTH TEES HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── South Tyneside & Sunderland
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── South Warwickshire
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── St. George's University Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Stockport NHS
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── STOCKPORT NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── SURREY AND SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Surrey & Sussex Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── TAMESIDE AND GLOSSOP INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Tameside Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Christie Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE CHRISTIE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Clatterbridge Cancer Centre
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE CLATTERBRIDGE CANCER CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Dudley Group
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE DUDLEY GROUP NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Hillingdon Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE HILLINGDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Newcastle upon Tyne Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE NEWCASTLE UPON TYNE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Princess Alexandra Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Queen Elizabeth Hospital King's Lynn
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Robert Jones & Agnes Hunt Orthopaedic Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE ROBERT JONES AND AGNES HUNT ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Rotherham
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE ROTHERHAM NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Royal Marsden
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL MARSDEN NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Royal Orthopaedic Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Royal Wolverhampton
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL WOLVERHAMPTON NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── The Walton Centre
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── THE WALTON CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Torbay and South Devon
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── United Lincolnshire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNITED LINCOLNSHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University College London Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY COLLEGE LONDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Birmingham
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Coventry & Warwickshire
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Dorset
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Derby and Burton
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Leicester
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF LEICESTER NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Morecambe Bay
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF MORECAMBE BAY NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of North Midlands
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospital Southampton
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITAL SOUTHAMPTON NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Plymouth
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Sussex NHS Foundation Trust
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Walsall Healthcare
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WALSALL HEALTHCARE NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WARRINGTON AND HALTON TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Warrington & Halton Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── West Hertfordshire Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WEST HERTFORDSHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── West Suffolk NHS
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WEST SUFFOLK NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Whittington Health
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WHITTINGTON HEALTH NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Wirral University Teaching Hospital
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Worcestershire Acute Hospitals
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Wrightington, Wigan & Leigh
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── Wye Valley
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── WYE VALLEY NHS TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   └── York Teaching Hospital
    │   │   │       └── e-coli_cases_countsByOnsetType
    │   │   ├── Sub-Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire ICB - 92G
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes ICB - M1J4Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull ICB - 15E
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country ICB - D2P2L
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire ICB - 15C
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 10Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 14Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 15A
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough ICB - 06H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01F
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01J
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01T
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01V
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01X
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 02E
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 12F
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 27D
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 99A
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and The Isles of Scilly ICB - 11N
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire ICB - B2M3M
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire ICB - 15M
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Devon ICB - 15N
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset ICB - 11J
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley ICB - D4U1Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire ICB - 11M
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00T
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00V
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01D
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01G
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01W
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02A
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 14L
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - 10R
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - D9Y0V
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire ICB - 18C
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06K
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06N
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 07H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 02Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03F
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03K
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 42D
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway ICB - 91Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00R
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00X
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01A
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01E
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01K
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02G
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02M
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 03W
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04C
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04V
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire ICB - 71E
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 06Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 07G
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99E
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99F
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99G
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney ICB - 26A
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire ICB - 78H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London ICB - 93C
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00L
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00N
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00P
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 01H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 13T
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 16C
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 84H
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 99C
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North East London ICB - A3A8R
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS North West London ICB - W2U3Z
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 02Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 52R
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin ICB - M2L0M
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset ICB - 11X
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South East London ICB - 72Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South West London ICB - 36L
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02P
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02X
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03L
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03N
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 04Y
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05D
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05G
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05Q
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05V
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05W
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06L
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06T
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 07K
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands ICB - 92A
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 09D
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 70F
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 97R
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 02T
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 03R
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 15F
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 36J
    │   │   │   │   └── e-coli_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire ICB - X2C4Y
    │   │   │       └── e-coli_cases_countsByOnsetType
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── East of England
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── London
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── North East
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── North West
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── South East
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── South West
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       ├── West Midlands
    │   │       │   └── e-coli_cases_countsByOnsetType
    │   │       └── Yorkshire and Humber
    │   │           └── e-coli_cases_countsByOnsetType
    │   ├── Klebsiella-spp
    │   │   ├── Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and the Isles of Scilly Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Devon Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East London Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North West London Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South East London Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South West London Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-on-Trent Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex Integrated Care Board
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire Integrated Care Board
    │   │   │       └── klebsiella-spp_cases_countsByOnsetType
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── klebsiella-spp_cases_countsByOnsetType
    │   │   ├── NHS Region
    │   │   │   ├── East of England
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── London
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Midlands
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North East and Yorkshire
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North West
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── South East
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   └── South West
    │   │   │       └── klebsiella-spp_cases_countsByOnsetType
    │   │   ├── NHS Trust
    │   │   │   ├── Airedale
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── AIREDALE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Alder Hey Children's
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Ashford & St Peter's Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Barking, Havering & Redbridge University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Barnsley Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BARNSLEY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Barts Health
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BARTS HEALTH NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Birmingham Women's and Children's
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Blackpool Teaching Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Bolton NHS
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BOLTON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Bradford Teaching Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Buckinghamshire Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── BUCKINGHAMSHIRE HEALTHCARE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── CALDERDALE AND HUDDERSFIELD NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Calderdale & Huddersfield
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Cambridge University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── CHELSEA AND WESTMINSTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Chelsea & Westminster Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Chesterfield Royal Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── CHESTERFIELD ROYAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Countess of Chester Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── COUNTESS OF CHESTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── COUNTY DURHAM AND DARLINGTON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── County Durham & Darlington
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Croydon Health Services
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── CROYDON HEALTH SERVICES NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── DARTFORD AND GRAVESHAM NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Dartford & Gravesham
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── DONCASTER AND BASSETLAW TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Doncaster & Bassetlaw Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Dorset County Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── DORSET COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EAST AND NORTH HERTFORDSHIRE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── East Cheshire
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EAST CHESHIRE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── East Kent Hospitals University
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── East Lancashire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EAST LANCASHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── East & North Hertfordshire
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── East Suffolk and North Essex
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EAST SUFFOLK AND NORTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── East Sussex Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EAST SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── EPSOM AND ST HELIER UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Epsom & St Helier University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Frimley Health
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── FRIMLEY HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Gateshead Health
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── GATESHEAD HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── George Eliot Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── GEORGE ELIOT HOSPITAL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Gloucestershire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Great Ormond Street Hospital for Children
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Great Western Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── GUY'S AND ST THOMAS' NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Guy's & St. Thomas'
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Hampshire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── HARROGATE AND DISTRICT NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Harrogate & District
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── HOMERTON HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Homerton University Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Hull University Teaching Hospitals NHS Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── HULL UNIVERSITY TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Imperial College Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── IMPERIAL COLLEGE HEALTHCARE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Isle of Wight
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ISLE OF WIGHT NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── James Paget University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── JAMES PAGET UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Kettering General Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── KETTERING GENERAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── King's College Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── KING'S COLLEGE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Kingston Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── KINGSTON HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Lancashire Teaching Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LANCASHIRE TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Leeds Teaching Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LEEDS TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LEWISHAM AND GREENWICH NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Lewisham & Greenwich
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL HEART AND CHEST HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Liverpool Heart & Chest Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Liverpool University Hospitals NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Liverpool Women's
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── London North West University Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── LONDON NORTH WEST UNIVERSITY HEALTHCARE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MAIDSTONE AND TUNBRIDGE WELLS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Maidstone & Tunbridge Wells
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Manchester University
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Medway
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MEDWAY NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MERSEY AND WEST LANCASHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Mid and South Essex NHS Foundation
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MID AND SOUTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Mid Cheshire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Mid Yorkshire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MID YORKSHIRE TEACHING NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Milton Keynes University Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MILTON KEYNES UNIVERSITY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Moorfields Eye Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── MOORFIELDS EYE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORFOLK AND NORWICH UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Norfolk & Norwich University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Northampton General Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTHAMPTON GENERAL HOSPITAL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North Bristol
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTH BRISTOL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTH CUMBRIA INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Northern Care Alliance NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTHERN LINCOLNSHIRE AND GOOLE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Northern Lincolnshire & Goole
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North Middlesex University Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTH MIDDLESEX UNIVERSITY HOSPITAL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTH TEES AND HARTLEPOOL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North Tees & Hartlepool
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Northumbria Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── North West Anglia
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NORTH WEST ANGLIA NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Nottingham University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Oxford University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── OXFORD UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Portsmouth Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── PORTSMOUTH HOSPITALS UNIVERSITY NATIONAL HEALTH SERVICE TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Queen Victoria Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── QUEEN VICTORIA HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal Berkshire
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL BERKSHIRE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal Cornwall Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL CORNWALL HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal Free London
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL FREE LONDON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal National Orthopaedic Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL NATIONAL ORTHOPAEDIC HOSPITAL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal Papworth Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal Surrey County Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Royal United Hospitals Bath
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ROYAL UNITED HOSPITALS BATH NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Salisbury
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SALISBURY NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Sandwell & West Birmingham Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Sheffield Children's
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Sheffield Teaching Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Sherwood Forest Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SHERWOOD FOREST HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Shrewsbury & Telford Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Somerset NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SOMERSET NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── South Tees Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SOUTH TEES HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── South Tyneside & Sunderland
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── South Warwickshire
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── St. George's University Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Stockport NHS
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── STOCKPORT NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── SURREY AND SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Surrey & Sussex Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── TAMESIDE AND GLOSSOP INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Tameside Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Christie Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE CHRISTIE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Clatterbridge Cancer Centre
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE CLATTERBRIDGE CANCER CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Dudley Group
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE DUDLEY GROUP NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Hillingdon Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE HILLINGDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Newcastle upon Tyne Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE NEWCASTLE UPON TYNE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Princess Alexandra Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Queen Elizabeth Hospital King's Lynn
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Robert Jones & Agnes Hunt Orthopaedic Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE ROBERT JONES AND AGNES HUNT ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Rotherham
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE ROTHERHAM NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Royal Marsden
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL MARSDEN NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Royal Orthopaedic Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Royal Wolverhampton
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL WOLVERHAMPTON NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── The Walton Centre
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── THE WALTON CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Torbay and South Devon
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── United Lincolnshire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNITED LINCOLNSHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University College London Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY COLLEGE LONDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Birmingham
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Coventry & Warwickshire
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Dorset
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Derby and Burton
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Leicester
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF LEICESTER NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Morecambe Bay
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF MORECAMBE BAY NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of North Midlands
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospital Southampton
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITAL SOUTHAMPTON NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Plymouth
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Sussex NHS Foundation Trust
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Walsall Healthcare
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WALSALL HEALTHCARE NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WARRINGTON AND HALTON TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Warrington & Halton Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── West Hertfordshire Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WEST HERTFORDSHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── West Suffolk NHS
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WEST SUFFOLK NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Whittington Health
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WHITTINGTON HEALTH NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Wirral University Teaching Hospital
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Worcestershire Acute Hospitals
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Wrightington, Wigan & Leigh
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── Wye Valley
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── WYE VALLEY NHS TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   └── York Teaching Hospital
    │   │   │       └── klebsiella-spp_cases_countsByOnsetType
    │   │   ├── Sub-Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire ICB - 92G
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes ICB - M1J4Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull ICB - 15E
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country ICB - D2P2L
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire ICB - 15C
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 10Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 14Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 15A
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough ICB - 06H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01F
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01J
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01T
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01V
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01X
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 02E
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 12F
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 27D
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 99A
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and The Isles of Scilly ICB - 11N
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire ICB - B2M3M
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire ICB - 15M
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Devon ICB - 15N
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset ICB - 11J
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley ICB - D4U1Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire ICB - 11M
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00T
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00V
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01D
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01G
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01W
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02A
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 14L
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - 10R
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - D9Y0V
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire ICB - 18C
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06K
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06N
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 07H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 02Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03F
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03K
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 42D
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway ICB - 91Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00R
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00X
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01A
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01E
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01K
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02G
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02M
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 03W
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04C
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04V
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire ICB - 71E
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 06Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 07G
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99E
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99F
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99G
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney ICB - 26A
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire ICB - 78H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London ICB - 93C
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00L
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00N
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00P
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 01H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 13T
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 16C
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 84H
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 99C
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North East London ICB - A3A8R
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS North West London ICB - W2U3Z
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 02Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 52R
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin ICB - M2L0M
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset ICB - 11X
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South East London ICB - 72Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South West London ICB - 36L
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02P
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02X
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03L
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03N
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 04Y
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05D
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05G
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05Q
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05V
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05W
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06L
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06T
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 07K
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands ICB - 92A
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 09D
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 70F
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 97R
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 02T
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 03R
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 15F
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 36J
    │   │   │   │   └── klebsiella-spp_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire ICB - X2C4Y
    │   │   │       └── klebsiella-spp_cases_countsByOnsetType
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── East of England
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── London
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── North East
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── North West
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── South East
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── South West
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       ├── West Midlands
    │   │       │   └── klebsiella-spp_cases_countsByOnsetType
    │   │       └── Yorkshire and Humber
    │   │           └── klebsiella-spp_cases_countsByOnsetType
    │   ├── MRSA
    │   │   ├── Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and the Isles of Scilly Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Devon Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East London Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North West London Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South East London Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South West London Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-on-Trent Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex Integrated Care Board
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire Integrated Care Board
    │   │   │       └── MRSA_cases_countsByOnsetType
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── MRSA_cases_countsByOnsetType
    │   │   ├── NHS Region
    │   │   │   ├── East of England
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── London
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Midlands
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North East and Yorkshire
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North West
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── South East
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   └── South West
    │   │   │       └── MRSA_cases_countsByOnsetType
    │   │   ├── NHS Trust
    │   │   │   ├── Airedale
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── AIREDALE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Alder Hey Children's
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Ashford & St Peter's Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Barking, Havering & Redbridge University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Barnsley Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BARNSLEY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Barts Health
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BARTS HEALTH NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Birmingham Women's and Children's
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Blackpool Teaching Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Bolton NHS
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BOLTON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Bradford Teaching Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Buckinghamshire Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── BUCKINGHAMSHIRE HEALTHCARE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── CALDERDALE AND HUDDERSFIELD NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Calderdale & Huddersfield
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Cambridge University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── CHELSEA AND WESTMINSTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Chelsea & Westminster Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Chesterfield Royal Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── CHESTERFIELD ROYAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Countess of Chester Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── COUNTESS OF CHESTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── COUNTY DURHAM AND DARLINGTON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── County Durham & Darlington
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Croydon Health Services
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── CROYDON HEALTH SERVICES NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── DARTFORD AND GRAVESHAM NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Dartford & Gravesham
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── DONCASTER AND BASSETLAW TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Doncaster & Bassetlaw Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Dorset County Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── DORSET COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EAST AND NORTH HERTFORDSHIRE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── East Cheshire
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EAST CHESHIRE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── East Kent Hospitals University
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── East Lancashire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EAST LANCASHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── East & North Hertfordshire
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── East Suffolk and North Essex
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EAST SUFFOLK AND NORTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── East Sussex Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EAST SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── EPSOM AND ST HELIER UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Epsom & St Helier University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Frimley Health
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── FRIMLEY HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Gateshead Health
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── GATESHEAD HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── George Eliot Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── GEORGE ELIOT HOSPITAL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Gloucestershire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Great Ormond Street Hospital for Children
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Great Western Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── GUY'S AND ST THOMAS' NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Guy's & St. Thomas'
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Hampshire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── HARROGATE AND DISTRICT NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Harrogate & District
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── HOMERTON HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Homerton University Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Hull University Teaching Hospitals NHS Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── HULL UNIVERSITY TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Imperial College Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── IMPERIAL COLLEGE HEALTHCARE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Isle of Wight
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ISLE OF WIGHT NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── James Paget University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── JAMES PAGET UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Kettering General Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── KETTERING GENERAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── King's College Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── KING'S COLLEGE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Kingston Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── KINGSTON HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Lancashire Teaching Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LANCASHIRE TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Leeds Teaching Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LEEDS TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LEWISHAM AND GREENWICH NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Lewisham & Greenwich
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL HEART AND CHEST HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Liverpool Heart & Chest Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Liverpool University Hospitals NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Liverpool Women's
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── London North West University Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── LONDON NORTH WEST UNIVERSITY HEALTHCARE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MAIDSTONE AND TUNBRIDGE WELLS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Maidstone & Tunbridge Wells
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Manchester University
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Medway
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MEDWAY NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MERSEY AND WEST LANCASHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Mid and South Essex NHS Foundation
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MID AND SOUTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Mid Cheshire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Mid Yorkshire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MID YORKSHIRE TEACHING NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Milton Keynes University Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MILTON KEYNES UNIVERSITY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Moorfields Eye Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── MOORFIELDS EYE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORFOLK AND NORWICH UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Norfolk & Norwich University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Northampton General Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTHAMPTON GENERAL HOSPITAL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North Bristol
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTH BRISTOL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTH CUMBRIA INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Northern Care Alliance NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTHERN LINCOLNSHIRE AND GOOLE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Northern Lincolnshire & Goole
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North Middlesex University Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTH MIDDLESEX UNIVERSITY HOSPITAL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTH TEES AND HARTLEPOOL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North Tees & Hartlepool
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Northumbria Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── North West Anglia
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NORTH WEST ANGLIA NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Nottingham University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Oxford University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── OXFORD UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Portsmouth Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── PORTSMOUTH HOSPITALS UNIVERSITY NATIONAL HEALTH SERVICE TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Queen Victoria Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── QUEEN VICTORIA HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal Berkshire
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL BERKSHIRE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal Cornwall Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL CORNWALL HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal Free London
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL FREE LONDON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal National Orthopaedic Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL NATIONAL ORTHOPAEDIC HOSPITAL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal Papworth Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal Surrey County Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Royal United Hospitals Bath
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL UNITED HOSPITALS BATH NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Salisbury
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SALISBURY NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Sandwell & West Birmingham Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Sheffield Children's
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Sheffield Teaching Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Sherwood Forest Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SHERWOOD FOREST HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Shrewsbury & Telford Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Somerset NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SOMERSET NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── South Tees Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SOUTH TEES HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── South Tyneside & Sunderland
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── South Warwickshire
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── St. George's University Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Stockport NHS
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── STOCKPORT NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── SURREY AND SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Surrey & Sussex Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── TAMESIDE AND GLOSSOP INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Tameside Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Christie Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE CHRISTIE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Clatterbridge Cancer Centre
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE CLATTERBRIDGE CANCER CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Dudley Group
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE DUDLEY GROUP NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Hillingdon Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE HILLINGDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Newcastle upon Tyne Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE NEWCASTLE UPON TYNE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Princess Alexandra Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Queen Elizabeth Hospital King's Lynn
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Robert Jones & Agnes Hunt Orthopaedic Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE ROBERT JONES AND AGNES HUNT ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Rotherham
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE ROTHERHAM NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Royal Marsden
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL MARSDEN NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Royal Orthopaedic Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Royal Wolverhampton
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL WOLVERHAMPTON NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── The Walton Centre
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── THE WALTON CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Torbay and South Devon
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── United Lincolnshire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNITED LINCOLNSHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University College London Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY COLLEGE LONDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Birmingham
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Coventry & Warwickshire
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Dorset
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Derby and Burton
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Leicester
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF LEICESTER NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Morecambe Bay
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF MORECAMBE BAY NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of North Midlands
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospital Southampton
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITAL SOUTHAMPTON NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Plymouth
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Sussex NHS Foundation Trust
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Walsall Healthcare
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WALSALL HEALTHCARE NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WARRINGTON AND HALTON TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Warrington & Halton Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── West Hertfordshire Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WEST HERTFORDSHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── West Suffolk NHS
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WEST SUFFOLK NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Whittington Health
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WHITTINGTON HEALTH NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Wirral University Teaching Hospital
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Worcestershire Acute Hospitals
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Wrightington, Wigan & Leigh
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── Wye Valley
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── WYE VALLEY NHS TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   └── York Teaching Hospital
    │   │   │       └── MRSA_cases_countsByOnsetType
    │   │   ├── Sub-Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire ICB - 92G
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes ICB - M1J4Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull ICB - 15E
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country ICB - D2P2L
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire ICB - 15C
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 10Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 14Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 15A
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough ICB - 06H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01F
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01J
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01T
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01V
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01X
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 02E
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 12F
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 27D
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 99A
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and The Isles of Scilly ICB - 11N
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire ICB - B2M3M
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire ICB - 15M
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Devon ICB - 15N
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset ICB - 11J
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley ICB - D4U1Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire ICB - 11M
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00T
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00V
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01D
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01G
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01W
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02A
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 14L
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - 10R
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - D9Y0V
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire ICB - 18C
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06K
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06N
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 07H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 02Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03F
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03K
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 42D
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway ICB - 91Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00R
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00X
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01A
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01E
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01K
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02G
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02M
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 03W
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04C
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04V
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire ICB - 71E
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 06Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 07G
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99E
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99F
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99G
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney ICB - 26A
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire ICB - 78H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London ICB - 93C
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00L
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00N
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00P
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 01H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 13T
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 16C
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 84H
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 99C
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East London ICB - A3A8R
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS North West London ICB - W2U3Z
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 02Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 52R
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin ICB - M2L0M
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset ICB - 11X
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South East London ICB - 72Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South West London ICB - 36L
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02P
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02X
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03L
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03N
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 04Y
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05D
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05G
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05Q
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05V
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05W
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06L
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06T
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 07K
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands ICB - 92A
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 09D
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 70F
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 97R
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 02T
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 03R
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 15F
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 36J
    │   │   │   │   └── MRSA_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire ICB - X2C4Y
    │   │   │       └── MRSA_cases_countsByOnsetType
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── East of England
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── London
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── North East
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── North West
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── South East
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── South West
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       ├── West Midlands
    │   │       │   └── MRSA_cases_countsByOnsetType
    │   │       └── Yorkshire and Humber
    │   │           └── MRSA_cases_countsByOnsetType
    │   ├── MSSA
    │   │   ├── Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and the Isles of Scilly Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Devon Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East London Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North West London Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South East London Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South West London Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-on-Trent Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex Integrated Care Board
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire Integrated Care Board
    │   │   │       └── MSSA_cases_countsByOnsetType
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── MSSA_cases_countsByOnsetType
    │   │   ├── NHS Region
    │   │   │   ├── East of England
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── London
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Midlands
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North East and Yorkshire
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North West
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── South East
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   └── South West
    │   │   │       └── MSSA_cases_countsByOnsetType
    │   │   ├── NHS Trust
    │   │   │   ├── Airedale
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── AIREDALE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Alder Hey Children's
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Ashford & St Peter's Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Barking, Havering & Redbridge University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Barnsley Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BARNSLEY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Barts Health
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BARTS HEALTH NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Birmingham Women's and Children's
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Blackpool Teaching Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Bolton NHS
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BOLTON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Bradford Teaching Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Buckinghamshire Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── BUCKINGHAMSHIRE HEALTHCARE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── CALDERDALE AND HUDDERSFIELD NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Calderdale & Huddersfield
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Cambridge University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── CHELSEA AND WESTMINSTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Chelsea & Westminster Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Chesterfield Royal Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── CHESTERFIELD ROYAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Countess of Chester Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── COUNTESS OF CHESTER HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── COUNTY DURHAM AND DARLINGTON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── County Durham & Darlington
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Croydon Health Services
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── CROYDON HEALTH SERVICES NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── DARTFORD AND GRAVESHAM NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Dartford & Gravesham
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── DONCASTER AND BASSETLAW TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Doncaster & Bassetlaw Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Dorset County Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── DORSET COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EAST AND NORTH HERTFORDSHIRE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── East Cheshire
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EAST CHESHIRE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── East Kent Hospitals University
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── East Lancashire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EAST LANCASHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── East & North Hertfordshire
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── East Suffolk and North Essex
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EAST SUFFOLK AND NORTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── East Sussex Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EAST SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── EPSOM AND ST HELIER UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Epsom & St Helier University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Frimley Health
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── FRIMLEY HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Gateshead Health
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── GATESHEAD HEALTH NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── George Eliot Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── GEORGE ELIOT HOSPITAL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Gloucestershire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Great Ormond Street Hospital for Children
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Great Western Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── GUY'S AND ST THOMAS' NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Guy's & St. Thomas'
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Hampshire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── HARROGATE AND DISTRICT NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Harrogate & District
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── HOMERTON HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Homerton University Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Hull University Teaching Hospitals NHS Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── HULL UNIVERSITY TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Imperial College Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── IMPERIAL COLLEGE HEALTHCARE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Isle of Wight
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ISLE OF WIGHT NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── James Paget University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── JAMES PAGET UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Kettering General Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── KETTERING GENERAL HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── King's College Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── KING'S COLLEGE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Kingston Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── KINGSTON HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Lancashire Teaching Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LANCASHIRE TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Leeds Teaching Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LEEDS TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LEWISHAM AND GREENWICH NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Lewisham & Greenwich
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL HEART AND CHEST HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Liverpool Heart & Chest Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Liverpool University Hospitals NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Liverpool Women's
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── London North West University Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── LONDON NORTH WEST UNIVERSITY HEALTHCARE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MAIDSTONE AND TUNBRIDGE WELLS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Maidstone & Tunbridge Wells
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Manchester University
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Medway
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MEDWAY NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MERSEY AND WEST LANCASHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Mid and South Essex NHS Foundation
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MID AND SOUTH ESSEX NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Mid Cheshire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Mid Yorkshire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MID YORKSHIRE TEACHING NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Milton Keynes University Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MILTON KEYNES UNIVERSITY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Moorfields Eye Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── MOORFIELDS EYE HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORFOLK AND NORWICH UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Norfolk & Norwich University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Northampton General Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTHAMPTON GENERAL HOSPITAL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North Bristol
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTH BRISTOL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTH CUMBRIA INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Northern Care Alliance NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTHERN LINCOLNSHIRE AND GOOLE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Northern Lincolnshire & Goole
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North Middlesex University Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTH MIDDLESEX UNIVERSITY HOSPITAL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTH TEES AND HARTLEPOOL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North Tees & Hartlepool
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Northumbria Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── North West Anglia
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NORTH WEST ANGLIA NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Nottingham University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Oxford University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── OXFORD UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Portsmouth Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── PORTSMOUTH HOSPITALS UNIVERSITY NATIONAL HEALTH SERVICE TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Queen Victoria Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── QUEEN VICTORIA HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal Berkshire
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL BERKSHIRE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal Cornwall Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL CORNWALL HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal Free London
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL FREE LONDON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal National Orthopaedic Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL NATIONAL ORTHOPAEDIC HOSPITAL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal Papworth Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal Surrey County Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Royal United Hospitals Bath
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ROYAL UNITED HOSPITALS BATH NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Salisbury
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SALISBURY NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Sandwell & West Birmingham Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Sheffield Children's
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Sheffield Teaching Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Sherwood Forest Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SHERWOOD FOREST HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Shrewsbury & Telford Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Somerset NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SOMERSET NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── South Tees Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SOUTH TEES HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── South Tyneside & Sunderland
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── South Warwickshire
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── St. George's University Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Stockport NHS
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── STOCKPORT NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── SURREY AND SUSSEX HEALTHCARE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Surrey & Sussex Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── TAMESIDE AND GLOSSOP INTEGRATED CARE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Tameside Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Christie Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE CHRISTIE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Clatterbridge Cancer Centre
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE CLATTERBRIDGE CANCER CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Dudley Group
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE DUDLEY GROUP NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Hillingdon Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE HILLINGDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Newcastle upon Tyne Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE NEWCASTLE UPON TYNE HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Princess Alexandra Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Queen Elizabeth Hospital King's Lynn
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Robert Jones & Agnes Hunt Orthopaedic Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE ROBERT JONES AND AGNES HUNT ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Rotherham
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE ROTHERHAM NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Royal Marsden
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL MARSDEN NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Royal Orthopaedic Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Royal Wolverhampton
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE ROYAL WOLVERHAMPTON NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── The Walton Centre
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── THE WALTON CENTRE NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Torbay and South Devon
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── United Lincolnshire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNITED LINCOLNSHIRE HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University College London Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY COLLEGE LONDON HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Birmingham
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Coventry & Warwickshire
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Dorset
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Derby and Burton
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Leicester
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF LEICESTER NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of Morecambe Bay
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF MORECAMBE BAY NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals of North Midlands
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospital Southampton
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITAL SOUTHAMPTON NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Plymouth
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── University Hospitals Sussex NHS Foundation Trust
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Walsall Healthcare
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WALSALL HEALTHCARE NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WARRINGTON AND HALTON TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Warrington & Halton Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── West Hertfordshire Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WEST HERTFORDSHIRE TEACHING HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── West Suffolk NHS
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WEST SUFFOLK NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Whittington Health
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WHITTINGTON HEALTH NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Wirral University Teaching Hospital
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Worcestershire Acute Hospitals
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Wrightington, Wigan & Leigh
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── Wye Valley
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── WYE VALLEY NHS TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   └── York Teaching Hospital
    │   │   │       └── MSSA_cases_countsByOnsetType
    │   │   ├── Sub-Integrated Care Board
    │   │   │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire ICB - 92G
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bedfordshire, Luton and Milton Keynes ICB - M1J4Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Birmingham and Solihull ICB - 15E
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Black Country ICB - D2P2L
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Bristol, North Somerset and South Gloucestershire ICB - 15C
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 10Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 14Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 15A
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cambridgeshire and Peterborough ICB - 06H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01F
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01J
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01T
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01V
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 01X
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 02E
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 12F
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 27D
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cheshire and Merseyside ICB - 99A
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Cornwall and The Isles of Scilly ICB - 11N
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Coventry and Warwickshire ICB - B2M3M
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Derby and Derbyshire ICB - 15M
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Devon ICB - 15N
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Dorset ICB - 11J
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Frimley ICB - D4U1Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Gloucestershire ICB - 11M
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00T
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00V
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 00Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01D
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01G
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01W
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 01Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02A
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 02H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Greater Manchester ICB - 14L
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - 10R
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hampshire and Isle of Wight ICB - D9Y0V
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Herefordshire and Worcestershire ICB - 18C
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06K
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 06N
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Hertfordshire and West Essex ICB - 07H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 02Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03F
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03K
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 03Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Humber and North Yorkshire ICB - 42D
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Kent and Medway ICB - 91Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00R
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 00X
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01A
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01E
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 01K
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02G
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lancashire and South Cumbria ICB - 02M
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 03W
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04C
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04V
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Lincolnshire ICB - 71E
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 06Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 07G
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99E
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99F
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Mid and South Essex ICB - 99G
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Norfolk and Waveney ICB - 26A
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Northamptonshire ICB - 78H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North Central London ICB - 93C
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00L
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00N
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 00P
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 01H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 13T
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 16C
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 84H
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East and North Cumbria ICB - 99C
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North East London ICB - A3A8R
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS North West London ICB - W2U3Z
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 02Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Nottingham and Nottinghamshire ICB - 52R
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Shropshire, Telford and Wrekin ICB - M2L0M
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Somerset ICB - 11X
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South East London ICB - 72Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South West London ICB - 36L
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02P
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 02X
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03L
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS South Yorkshire ICB - 03N
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 04Y
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05D
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05G
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05Q
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05V
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05W
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06L
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 06T
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Suffolk and North East Essex ICB - 07K
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Surrey Heartlands ICB - 92A
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 09D
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 70F
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS Sussex ICB - 97R
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 02T
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 03R
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 15F
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   ├── NHS West Yorkshire ICB - 36J
    │   │   │   │   └── MSSA_cases_countsByOnsetType
    │   │   │   └── NHS West Yorkshire ICB - X2C4Y
    │   │   │       └── MSSA_cases_countsByOnsetType
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── East of England
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── London
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── North East
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── North West
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── South East
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── South West
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       ├── West Midlands
    │   │       │   └── MSSA_cases_countsByOnsetType
    │   │       └── Yorkshire and Humber
    │   │           └── MSSA_cases_countsByOnsetType
    │   └── Pseudomonas-aeruginosa
    │       ├── Integrated Care Board
    │       │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Birmingham and Solihull Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Black Country Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cambridgeshire and Peterborough Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cornwall and the Isles of Scilly Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Coventry and Warwickshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Derby and Derbyshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Devon Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Dorset Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Frimley Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Gloucestershire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hampshire and Isle of Wight Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Herefordshire and Worcestershire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Kent and Medway Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lincolnshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Norfolk and Waveney Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Northamptonshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North Central London Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East London Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North West London Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Nottingham and Nottinghamshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Shropshire, Telford and Wrekin Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Somerset Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South East London Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South West London Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-on-Trent Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Surrey Heartlands Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Sussex Integrated Care Board
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   └── NHS West Yorkshire Integrated Care Board
    │       │       └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       ├── Nation
    │       │   └── England
    │       │       └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       ├── NHS Region
    │       │   ├── East of England
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── London
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Midlands
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North East and Yorkshire
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North West
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── South East
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   └── South West
    │       │       └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       ├── NHS Trust
    │       │   ├── Airedale
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── AIREDALE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Alder Hey Children's
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Ashford & St Peter's Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Barking, Havering & Redbridge University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Barnsley Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BARNSLEY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Barts Health
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BARTS HEALTH NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Birmingham Women's and Children's
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Blackpool Teaching Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Bolton NHS
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BOLTON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Bradford Teaching Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Buckinghamshire Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── BUCKINGHAMSHIRE HEALTHCARE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── CALDERDALE AND HUDDERSFIELD NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Calderdale & Huddersfield
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Cambridge University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── CHELSEA AND WESTMINSTER HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Chelsea & Westminster Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Chesterfield Royal Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── CHESTERFIELD ROYAL HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Countess of Chester Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── COUNTESS OF CHESTER HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── COUNTY DURHAM AND DARLINGTON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── County Durham & Darlington
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Croydon Health Services
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── CROYDON HEALTH SERVICES NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── DARTFORD AND GRAVESHAM NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Dartford & Gravesham
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── DONCASTER AND BASSETLAW TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Doncaster & Bassetlaw Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Dorset County Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── DORSET COUNTY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EAST AND NORTH HERTFORDSHIRE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── East Cheshire
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EAST CHESHIRE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── East Kent Hospitals University
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── East Lancashire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EAST LANCASHIRE HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── East & North Hertfordshire
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── East Suffolk and North Essex
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EAST SUFFOLK AND NORTH ESSEX NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── East Sussex Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EAST SUSSEX HEALTHCARE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── EPSOM AND ST HELIER UNIVERSITY HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Epsom & St Helier University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Frimley Health
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── FRIMLEY HEALTH NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Gateshead Health
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── GATESHEAD HEALTH NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── George Eliot Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── GEORGE ELIOT HOSPITAL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Gloucestershire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Great Ormond Street Hospital for Children
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Great Western Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── GUY'S AND ST THOMAS' NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Guy's & St. Thomas'
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Hampshire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── HARROGATE AND DISTRICT NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Harrogate & District
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── HOMERTON HEALTHCARE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Homerton University Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Hull University Teaching Hospitals NHS Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── HULL UNIVERSITY TEACHING HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Imperial College Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── IMPERIAL COLLEGE HEALTHCARE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Isle of Wight
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ISLE OF WIGHT NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── James Paget University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── JAMES PAGET UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Kettering General Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── KETTERING GENERAL HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── King's College Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── KING'S COLLEGE HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Kingston Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── KINGSTON HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Lancashire Teaching Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LANCASHIRE TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Leeds Teaching Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LEEDS TEACHING HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LEWISHAM AND GREENWICH NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Lewisham & Greenwich
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LIVERPOOL HEART AND CHEST HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Liverpool Heart & Chest Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Liverpool University Hospitals NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Liverpool Women's
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── London North West University Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── LONDON NORTH WEST UNIVERSITY HEALTHCARE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MAIDSTONE AND TUNBRIDGE WELLS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Maidstone & Tunbridge Wells
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Manchester University
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Medway
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MEDWAY NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MERSEY AND WEST LANCASHIRE TEACHING HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Mid and South Essex NHS Foundation
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MID AND SOUTH ESSEX NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Mid Cheshire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Mid Yorkshire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MID YORKSHIRE TEACHING NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Milton Keynes University Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MILTON KEYNES UNIVERSITY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Moorfields Eye Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── MOORFIELDS EYE HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORFOLK AND NORWICH UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Norfolk & Norwich University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Northampton General Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTHAMPTON GENERAL HOSPITAL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North Bristol
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTH BRISTOL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTH CUMBRIA INTEGRATED CARE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Northern Care Alliance NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTHERN LINCOLNSHIRE AND GOOLE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Northern Lincolnshire & Goole
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North Middlesex University Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTH MIDDLESEX UNIVERSITY HOSPITAL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTH TEES AND HARTLEPOOL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North Tees & Hartlepool
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Northumbria Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── North West Anglia
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NORTH WEST ANGLIA NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Nottingham University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Oxford University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── OXFORD UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Portsmouth Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── PORTSMOUTH HOSPITALS UNIVERSITY NATIONAL HEALTH SERVICE TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Queen Victoria Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── QUEEN VICTORIA HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal Berkshire
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL BERKSHIRE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal Cornwall Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL CORNWALL HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal Free London
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL FREE LONDON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal National Orthopaedic Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL NATIONAL ORTHOPAEDIC HOSPITAL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal Papworth Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal Surrey County Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Royal United Hospitals Bath
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ROYAL UNITED HOSPITALS BATH NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Salisbury
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SALISBURY NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Sandwell & West Birmingham Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Sheffield Children's
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Sheffield Teaching Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Sherwood Forest Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SHERWOOD FOREST HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Shrewsbury & Telford Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Somerset NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SOMERSET NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── South Tees Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SOUTH TEES HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── South Tyneside & Sunderland
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── South Warwickshire
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── St. George's University Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Stockport NHS
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── STOCKPORT NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── SURREY AND SUSSEX HEALTHCARE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Surrey & Sussex Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── TAMESIDE AND GLOSSOP INTEGRATED CARE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Tameside Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Christie Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE CHRISTIE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Clatterbridge Cancer Centre
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE CLATTERBRIDGE CANCER CENTRE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Dudley Group
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE DUDLEY GROUP NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Hillingdon Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE HILLINGDON HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Newcastle upon Tyne Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE NEWCASTLE UPON TYNE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Princess Alexandra Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Queen Elizabeth Hospital King's Lynn
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Robert Jones & Agnes Hunt Orthopaedic Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE ROBERT JONES AND AGNES HUNT ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Rotherham
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE ROTHERHAM NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Royal Marsden
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE ROYAL MARSDEN NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Royal Orthopaedic Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Royal Wolverhampton
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE ROYAL WOLVERHAMPTON NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── The Walton Centre
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── THE WALTON CENTRE NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Torbay and South Devon
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── United Lincolnshire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNITED LINCOLNSHIRE HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University College London Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY COLLEGE LONDON HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals Birmingham
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals Coventry & Warwickshire
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals Dorset
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals of Derby and Burton
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals of Leicester
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF LEICESTER NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals of Morecambe Bay
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF MORECAMBE BAY NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals of North Midlands
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospital Southampton
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITAL SOUTHAMPTON NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals Plymouth
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── University Hospitals Sussex NHS Foundation Trust
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Walsall Healthcare
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WALSALL HEALTHCARE NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WARRINGTON AND HALTON TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Warrington & Halton Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── West Hertfordshire Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WEST HERTFORDSHIRE TEACHING HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── West Suffolk NHS
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WEST SUFFOLK NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Whittington Health
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WHITTINGTON HEALTH NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Wirral University Teaching Hospital
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Worcestershire Acute Hospitals
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Wrightington, Wigan & Leigh
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── Wye Valley
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── WYE VALLEY NHS TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   └── York Teaching Hospital
    │       │       └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       ├── Sub-Integrated Care Board
    │       │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire ICB - 92G
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Bedfordshire, Luton and Milton Keynes ICB - M1J4Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Birmingham and Solihull ICB - 15E
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Black Country ICB - D2P2L
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Bristol, North Somerset and South Gloucestershire ICB - 15C
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 10Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 14Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 15A
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cambridgeshire and Peterborough ICB - 06H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01F
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01J
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01T
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01V
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01X
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 02E
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 12F
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 27D
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 99A
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Cornwall and The Isles of Scilly ICB - 11N
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Coventry and Warwickshire ICB - B2M3M
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Derby and Derbyshire ICB - 15M
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Devon ICB - 15N
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Dorset ICB - 11J
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Frimley ICB - D4U1Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Gloucestershire ICB - 11M
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 00T
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 00V
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 00Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01D
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01G
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01W
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 02A
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 02H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 14L
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hampshire and Isle of Wight ICB - 10R
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hampshire and Isle of Wight ICB - D9Y0V
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Herefordshire and Worcestershire ICB - 18C
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex ICB - 06K
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex ICB - 06N
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex ICB - 07H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 02Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03F
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03K
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 42D
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Kent and Medway ICB - 91Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 00Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 00R
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 00X
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 01A
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 01E
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 01K
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 02G
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 02M
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland ICB - 03W
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04C
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04V
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Lincolnshire ICB - 71E
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 06Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 07G
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 99E
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 99F
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 99G
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Norfolk and Waveney ICB - 26A
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Northamptonshire ICB - 78H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North Central London ICB - 93C
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 00L
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 00N
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 00P
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 01H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 13T
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 16C
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 84H
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 99C
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North East London ICB - A3A8R
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS North West London ICB - W2U3Z
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Nottingham and Nottinghamshire ICB - 02Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Nottingham and Nottinghamshire ICB - 52R
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Shropshire, Telford and Wrekin ICB - M2L0M
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Somerset ICB - 11X
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South East London ICB - 72Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South West London ICB - 36L
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 02P
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 02X
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 03L
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 03N
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 04Y
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05D
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05G
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05Q
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05V
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05W
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex ICB - 06L
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex ICB - 06T
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex ICB - 07K
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Surrey Heartlands ICB - 92A
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Sussex ICB - 09D
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Sussex ICB - 70F
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS Sussex ICB - 97R
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 02T
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 03R
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 15F
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 36J
    │       │   │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       │   └── NHS West Yorkshire ICB - X2C4Y
    │       │       └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │       └── UKHSA Region
    │           ├── East Midlands
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── East of England
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── London
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── North East
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── North West
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── South East
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── South West
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           ├── West Midlands
    │           │   └── pseudomonas-aeruginosa_cases_countsByOnsetType
    │           └── Yorkshire and Humber
    │               └── pseudomonas-aeruginosa_cases_countsByOnsetType
    ├── gastrointestinal
    │   └── C-difficile
    │       ├── Integrated Care Board
    │       │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Bedfordshire, Luton and Milton Keynes Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Birmingham and Solihull Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Black Country Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Bristol, North Somerset and South Gloucestershire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cambridgeshire and Peterborough Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cornwall and the Isles of Scilly Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Coventry and Warwickshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Derby and Derbyshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Devon Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Dorset Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Frimley Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Gloucestershire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hampshire and Isle of Wight Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Herefordshire and Worcestershire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Kent and Medway Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lincolnshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Norfolk and Waveney Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Northamptonshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North Central London Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East London Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North West London Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Nottingham and Nottinghamshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Shropshire, Telford and Wrekin Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Somerset Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South East London Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South West London Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-on-Trent Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Surrey Heartlands Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Sussex Integrated Care Board
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   └── NHS West Yorkshire Integrated Care Board
    │       │       └── c-difficile_cases_countsByOnsetType
    │       ├── Nation
    │       │   └── England
    │       │       └── c-difficile_cases_countsByOnsetType
    │       ├── NHS Region
    │       │   ├── East of England
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── London
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Midlands
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North East and Yorkshire
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North West
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── South East
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   └── South West
    │       │       └── c-difficile_cases_countsByOnsetType
    │       ├── NHS Trust
    │       │   ├── Airedale
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── AIREDALE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Alder Hey Children's
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ALDER HEY CHILDREN'S NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ASHFORD AND ST PETER'S HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Ashford & St Peter's Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BARKING, HAVERING AND REDBRIDGE UNIVERSITY HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Barking, Havering & Redbridge University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Barnsley Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BARNSLEY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Barts Health
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BARTS HEALTH NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BEDFORDSHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Birmingham Women's and Children's
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BIRMINGHAM WOMEN'S AND CHILDREN'S NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Blackpool Teaching Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BLACKPOOL TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Bolton NHS
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BOLTON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Bradford Teaching Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BRADFORD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Buckinghamshire Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── BUCKINGHAMSHIRE HEALTHCARE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── CALDERDALE AND HUDDERSFIELD NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Calderdale & Huddersfield
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Cambridge University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── CAMBRIDGE UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── CHELSEA AND WESTMINSTER HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Chelsea & Westminster Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Chesterfield Royal Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── CHESTERFIELD ROYAL HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Countess of Chester Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── COUNTESS OF CHESTER HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── COUNTY DURHAM AND DARLINGTON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── County Durham & Darlington
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Croydon Health Services
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── CROYDON HEALTH SERVICES NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── DARTFORD AND GRAVESHAM NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Dartford & Gravesham
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── DONCASTER AND BASSETLAW TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Doncaster & Bassetlaw Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Dorset County Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── DORSET COUNTY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EAST AND NORTH HERTFORDSHIRE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── East Cheshire
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EAST CHESHIRE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── East Kent Hospitals University
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EAST KENT HOSPITALS UNIVERSITY NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── East Lancashire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EAST LANCASHIRE HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── East & North Hertfordshire
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── East Suffolk and North Essex
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EAST SUFFOLK AND NORTH ESSEX NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── East Sussex Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EAST SUSSEX HEALTHCARE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── EPSOM AND ST HELIER UNIVERSITY HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Epsom & St Helier University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Frimley Health
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── FRIMLEY HEALTH NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Gateshead Health
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── GATESHEAD HEALTH NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── George Eliot Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── GEORGE ELIOT HOSPITAL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Gloucestershire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── GLOUCESTERSHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Great Ormond Street Hospital for Children
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── GREAT ORMOND STREET HOSPITAL FOR CHILDREN NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Great Western Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── GREAT WESTERN HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── GUY'S AND ST THOMAS' NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Guy's & St. Thomas'
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Hampshire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── HAMPSHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── HARROGATE AND DISTRICT NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Harrogate & District
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── HOMERTON HEALTHCARE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Homerton University Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Hull University Teaching Hospitals NHS Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── HULL UNIVERSITY TEACHING HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Imperial College Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── IMPERIAL COLLEGE HEALTHCARE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Isle of Wight
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ISLE OF WIGHT NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── James Paget University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── JAMES PAGET UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Kettering General Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── KETTERING GENERAL HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── King's College Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── KING'S COLLEGE HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Kingston Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── KINGSTON HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Lancashire Teaching Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LANCASHIRE TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Leeds Teaching Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LEEDS TEACHING HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LEWISHAM AND GREENWICH NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Lewisham & Greenwich
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LIVERPOOL HEART AND CHEST HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Liverpool Heart & Chest Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Liverpool University Hospitals NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LIVERPOOL UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Liverpool Women's
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LIVERPOOL WOMEN'S NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── London North West University Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── LONDON NORTH WEST UNIVERSITY HEALTHCARE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MAIDSTONE AND TUNBRIDGE WELLS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Maidstone & Tunbridge Wells
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Manchester University
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MANCHESTER UNIVERSITY NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Medway
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MEDWAY NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MERSEY AND WEST LANCASHIRE TEACHING HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Mid and South Essex NHS Foundation
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MID AND SOUTH ESSEX NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Mid Cheshire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MID CHESHIRE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Mid Yorkshire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MID YORKSHIRE TEACHING NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Milton Keynes University Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MILTON KEYNES UNIVERSITY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Moorfields Eye Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── MOORFIELDS EYE HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORFOLK AND NORWICH UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Norfolk & Norwich University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Northampton General Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTHAMPTON GENERAL HOSPITAL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North Bristol
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTH BRISTOL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTH CUMBRIA INTEGRATED CARE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Northern Care Alliance NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTHERN CARE ALLIANCE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTHERN LINCOLNSHIRE AND GOOLE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Northern Lincolnshire & Goole
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North Middlesex University Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTH MIDDLESEX UNIVERSITY HOSPITAL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTH TEES AND HARTLEPOOL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North Tees & Hartlepool
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Northumbria Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTHUMBRIA HEALTHCARE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── North West Anglia
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NORTH WEST ANGLIA NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Nottingham University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NOTTINGHAM UNIVERSITY HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Oxford University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── OXFORD UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Portsmouth Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── PORTSMOUTH HOSPITALS UNIVERSITY NATIONAL HEALTH SERVICE TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Queen Victoria Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── QUEEN VICTORIA HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal Berkshire
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL BERKSHIRE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal Cornwall Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL CORNWALL HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL DEVON UNIVERSITY HEALTHCARE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal Free London
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL FREE LONDON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal National Orthopaedic Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL NATIONAL ORTHOPAEDIC HOSPITAL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal Papworth Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL PAPWORTH HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal Surrey County Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL SURREY COUNTY HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Royal United Hospitals Bath
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ROYAL UNITED HOSPITALS BATH NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Salisbury
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SALISBURY NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SANDWELL AND WEST BIRMINGHAM HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Sandwell & West Birmingham Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Sheffield Children's
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SHEFFIELD CHILDREN'S NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Sheffield Teaching Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SHEFFIELD TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Sherwood Forest Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SHERWOOD FOREST HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Shrewsbury & Telford Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Somerset NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SOMERSET NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── South Tees Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SOUTH TEES HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SOUTH TYNESIDE AND SUNDERLAND NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── South Tyneside & Sunderland
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── South Warwickshire
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SOUTH WARWICKSHIRE UNIVERSITY NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── St. George's University Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── ST GEORGE'S UNIVERSITY HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Stockport NHS
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── STOCKPORT NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── SURREY AND SUSSEX HEALTHCARE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Surrey & Sussex Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── TAMESIDE AND GLOSSOP INTEGRATED CARE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Tameside Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Christie Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE CHRISTIE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Clatterbridge Cancer Centre
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE CLATTERBRIDGE CANCER CENTRE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Dudley Group
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE DUDLEY GROUP NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Hillingdon Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE HILLINGDON HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Newcastle upon Tyne Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE NEWCASTLE UPON TYNE HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Princess Alexandra Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE PRINCESS ALEXANDRA HOSPITAL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Queen Elizabeth Hospital King's Lynn
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE QUEEN ELIZABETH HOSPITAL, KING'S LYNN, NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Robert Jones & Agnes Hunt Orthopaedic Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE ROBERT JONES AND AGNES HUNT ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Rotherham
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE ROTHERHAM NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Royal Marsden
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE ROYAL MARSDEN NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Royal Orthopaedic Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE ROYAL ORTHOPAEDIC HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Royal Wolverhampton
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE ROYAL WOLVERHAMPTON NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE SHREWSBURY AND TELFORD HOSPITAL NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── The Walton Centre
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── THE WALTON CENTRE NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Torbay and South Devon
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── TORBAY AND SOUTH DEVON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── United Lincolnshire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNITED LINCOLNSHIRE HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University College London Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY COLLEGE LONDON HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals Birmingham
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS BIRMINGHAM NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS BRISTOL AND WESTON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS COVENTRY AND WARWICKSHIRE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals Coventry & Warwickshire
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals Dorset
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS DORSET NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals of Derby and Burton
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF DERBY AND BURTON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals of Leicester
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF LEICESTER NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals of Morecambe Bay
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF MORECAMBE BAY NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals of North Midlands
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS OF NORTH MIDLANDS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospital Southampton
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITAL SOUTHAMPTON NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals Plymouth
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS PLYMOUTH NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── University Hospitals Sussex NHS Foundation Trust
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── UNIVERSITY HOSPITALS SUSSEX NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Walsall Healthcare
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WALSALL HEALTHCARE NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WARRINGTON AND HALTON TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Warrington & Halton Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── West Hertfordshire Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WEST HERTFORDSHIRE TEACHING HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── West Suffolk NHS
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WEST SUFFOLK NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Whittington Health
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WHITTINGTON HEALTH NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Wirral University Teaching Hospital
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WIRRAL UNIVERSITY TEACHING HOSPITAL NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Worcestershire Acute Hospitals
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WORCESTERSHIRE ACUTE HOSPITALS NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WRIGHTINGTON, WIGAN AND LEIGH NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Wrightington, Wigan & Leigh
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── Wye Valley
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── WYE VALLEY NHS TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── YORK AND SCARBOROUGH TEACHING HOSPITALS NHS FOUNDATION TRUST
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   └── York Teaching Hospital
    │       │       └── c-difficile_cases_countsByOnsetType
    │       ├── Sub-Integrated Care Board
    │       │   ├── NHS Bath and North East Somerset, Swindon and Wiltshire ICB - 92G
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Bedfordshire, Luton and Milton Keynes ICB - M1J4Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Birmingham and Solihull ICB - 15E
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Black Country ICB - D2P2L
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Bristol, North Somerset and South Gloucestershire ICB - 15C
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 10Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 14Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Buckinghamshire, Oxfordshire and Berkshire West ICB - 15A
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cambridgeshire and Peterborough ICB - 06H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01F
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01J
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01T
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01V
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 01X
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 02E
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 12F
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 27D
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cheshire and Merseyside ICB - 99A
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Cornwall and The Isles of Scilly ICB - 11N
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Coventry and Warwickshire ICB - B2M3M
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Derby and Derbyshire ICB - 15M
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Devon ICB - 15N
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Dorset ICB - 11J
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Frimley ICB - D4U1Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Gloucestershire ICB - 11M
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 00T
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 00V
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 00Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01D
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01G
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01W
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 01Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 02A
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 02H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Greater Manchester ICB - 14L
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hampshire and Isle of Wight ICB - 10R
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hampshire and Isle of Wight ICB - D9Y0V
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Herefordshire and Worcestershire ICB - 18C
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex ICB - 06K
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex ICB - 06N
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Hertfordshire and West Essex ICB - 07H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 02Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03F
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03K
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 03Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Humber and North Yorkshire ICB - 42D
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Kent and Medway ICB - 91Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 00Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 00R
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 00X
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 01A
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 01E
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 01K
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 02G
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lancashire and South Cumbria ICB - 02M
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland ICB - 03W
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04C
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Leicester, Leicestershire and Rutland ICB - 04V
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Lincolnshire ICB - 71E
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 06Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 07G
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 99E
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 99F
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Mid and South Essex ICB - 99G
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Norfolk and Waveney ICB - 26A
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Northamptonshire ICB - 78H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North Central London ICB - 93C
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 00L
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 00N
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 00P
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 01H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 13T
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 16C
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 84H
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East and North Cumbria ICB - 99C
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North East London ICB - A3A8R
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS North West London ICB - W2U3Z
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Nottingham and Nottinghamshire ICB - 02Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Nottingham and Nottinghamshire ICB - 52R
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Shropshire, Telford and Wrekin ICB - M2L0M
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Somerset ICB - 11X
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South East London ICB - 72Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South West London ICB - 36L
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 02P
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 02X
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 03L
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS South Yorkshire ICB - 03N
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 04Y
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05D
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05G
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05Q
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05V
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Staffordshire and Stoke-On-Trent ICB - 05W
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex ICB - 06L
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex ICB - 06T
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Suffolk and North East Essex ICB - 07K
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Surrey Heartlands ICB - 92A
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Sussex ICB - 09D
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Sussex ICB - 70F
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS Sussex ICB - 97R
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 02T
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 03R
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 15F
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   ├── NHS West Yorkshire ICB - 36J
    │       │   │   └── c-difficile_cases_countsByOnsetType
    │       │   └── NHS West Yorkshire ICB - X2C4Y
    │       │       └── c-difficile_cases_countsByOnsetType
    │       └── UKHSA Region
    │           ├── East Midlands
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── East of England
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── London
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── North East
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── North West
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── South East
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── South West
    │           │   └── c-difficile_cases_countsByOnsetType
    │           ├── West Midlands
    │           │   └── c-difficile_cases_countsByOnsetType
    │           └── Yorkshire and Humber
    │               └── c-difficile_cases_countsByOnsetType
    ├── respiratory
    │   ├── Adenovirus
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── adenovirus_testing_positivityByWeek
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── East of England
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── London
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── North East
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── North West
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── South East
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── South West
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       ├── West Midlands
    │   │       │   └── adenovirus_testing_positivityByWeek
    │   │       └── Yorkshire and Humber
    │   │           └── adenovirus_testing_positivityByWeek
    │   ├── COVID-19
    │   │   ├── Government Office Region
    │   │   │   ├── East Midlands
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── East of England
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── London
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── North East
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── North West
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── South East
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── South West
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   ├── West Midlands
    │   │   │   │   ├── COVID-19_deaths_ONSByDay
    │   │   │   │   ├── COVID-19_deaths_ONSRegByWeek
    │   │   │   │   └── COVID-19_deaths_ONSRollingMean
    │   │   │   └── Yorkshire and The Humber
    │   │   │       ├── COVID-19_deaths_ONSByDay
    │   │   │       ├── COVID-19_deaths_ONSRegByWeek
    │   │   │       └── COVID-19_deaths_ONSRollingMean
    │   │   ├── Lower Tier Local Authority
    │   │   │   ├── Adur
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Allerdale
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Amber Valley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Arun
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Ashfield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Ashford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Babergh
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Barking and Dagenham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Barnet
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Barnsley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Barrow-in-Furness
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Basildon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Basingstoke and Deane
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bassetlaw
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bath and North East Somerset
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bedford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bexley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Birmingham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Blaby
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Blackburn with Darwen
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Blackpool
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bolsover
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bolton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Boston
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bournemouth, Christchurch and Poole
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bracknell Forest
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bradford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Braintree
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Breckland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Brent
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Brentwood
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Brighton and Hove
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bristol, City of
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Broadland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bromley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bromsgrove
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Broxbourne
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Broxtowe
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Buckinghamshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Burnley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Bury
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Calderdale
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cambridge
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Camden
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cannock Chase
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Canterbury
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Carlisle
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Castle Point
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Central Bedfordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Charnwood
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Chelmsford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cheltenham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cherwell
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cheshire East
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cheshire West and Chester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Chesterfield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Chichester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Chorley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── City of London
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Colchester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Copeland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Corby
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Cornwall
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Cornwall and Isles of Scilly
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Cotswold
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── County Durham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Coventry
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Craven
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Crawley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Croydon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Dacorum
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Darlington
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Dartford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Daventry
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Derby
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Derbyshire Dales
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Doncaster
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Dorset
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Dover
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Dudley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Ealing
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Eastbourne
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Cambridgeshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Devon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Hampshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Hertfordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Eastleigh
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Lindsey
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Northamptonshire
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── East Riding of Yorkshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Staffordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East Suffolk
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Eden
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Elmbridge
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Enfield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Epping Forest
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Epsom and Ewell
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Erewash
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Exeter
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Fareham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Fenland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Folkestone and Hythe
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Forest of Dean
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Fylde
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Gateshead
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Gedling
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Gloucester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Gosport
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Gravesham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Great Yarmouth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Greenwich
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Guildford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hackney
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Hackney and City of London
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Halton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hambleton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hammersmith and Fulham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Harborough
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Haringey
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Harlow
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Harrogate
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Harrow
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hart
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hartlepool
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hastings
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Havant
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Havering
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Herefordshire, County of
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hertsmere
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── High Peak
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hillingdon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hinckley and Bosworth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Horsham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hounslow
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Huntingdonshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Hyndburn
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Ipswich
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Isle of Wight
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Isles of Scilly
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Islington
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Kensington and Chelsea
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Kettering
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── King's Lynn and West Norfolk
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Kingston upon Hull, City of
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Kingston upon Thames
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Kirklees
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Knowsley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Lambeth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Lancaster
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Leeds
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Leicester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Lewes
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Lewisham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Lichfield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Lincoln
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Liverpool
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Luton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Maidstone
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Maldon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Malvern Hills
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Manchester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Mansfield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Medway
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Melton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Mendip
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Merton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Mid Devon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Middlesbrough
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Mid Suffolk
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Mid Sussex
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Milton Keynes
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Mole Valley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Newark and Sherwood
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Newcastle-under-Lyme
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Newcastle upon Tyne
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── New Forest
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Newham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Northampton
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── North Devon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North East Derbyshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North East Lincolnshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Hertfordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Kesteven
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Lincolnshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Norfolk
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Northamptonshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Somerset
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Tyneside
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Northumberland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North Warwickshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North West Leicestershire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Norwich
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Nottingham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Nuneaton and Bedworth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Oadby and Wigston
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Oldham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Oxford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Pendle
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Peterborough
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Plymouth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Portsmouth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Preston
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Reading
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Redbridge
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Redcar and Cleveland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Redditch
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Reigate and Banstead
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Ribble Valley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Richmondshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Richmond upon Thames
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rochdale
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rochford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rossendale
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rother
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rotherham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rugby
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Runnymede
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rushcliffe
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rushmoor
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Rutland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Ryedale
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Salford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sandwell
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Scarborough
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sedgemoor
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sefton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Selby
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sevenoaks
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sheffield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Shropshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Slough
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Solihull
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Somerset West and Taunton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Southampton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Cambridgeshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Derbyshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Southend-on-Sea
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Gloucestershire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Hams
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Holland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Kesteven
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Lakeland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Norfolk
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Northamptonshire
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── South Oxfordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Ribble
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Somerset
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Staffordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South Tyneside
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Southwark
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Spelthorne
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stafford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Staffordshire Moorlands
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── St Albans
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stevenage
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── St. Helens
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stockport
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stockton-on-Tees
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stoke-on-Trent
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stratford-on-Avon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Stroud
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sunderland
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Surrey Heath
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Sutton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Swale
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Swindon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tameside
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tamworth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tandridge
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Teignbridge
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Telford and Wrekin
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tendring
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Test Valley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tewkesbury
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Thanet
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Three Rivers
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Thurrock
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tonbridge and Malling
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Torbay
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Torridge
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tower Hamlets
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Trafford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Tunbridge Wells
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Uttlesford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Vale of White Horse
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wakefield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Walsall
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Waltham Forest
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wandsworth
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Warrington
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Warwick
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Watford
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Waverley
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wealden
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wellingborough
    │   │   │   │   └── COVID-19_deaths_ONSByWeek
    │   │   │   ├── Welwyn Hatfield
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Berkshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Devon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Lancashire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Lindsey
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Westminster
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Northamptonshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Oxfordshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Suffolk
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wigan
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wiltshire
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Winchester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Windsor and Maidenhead
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wirral
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Woking
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wokingham
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wolverhampton
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Worcester
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Worthing
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wychavon
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wyre
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── Wyre Forest
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_deaths_ONSByWeek
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   └── York
    │   │   │       ├── COVID-19_cases_casesByDay
    │   │   │       ├── COVID-19_cases_countRollingMean
    │   │   │       ├── COVID-19_cases_rateRollingMean
    │   │   │       ├── COVID-19_deaths_ONSByWeek
    │   │   │       ├── COVID-19_testing_PCRcountByDay
    │   │   │       ├── COVID-19_testing_positivity7DayRolling
    │   │   │       ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │       └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       ├── COVID-19_cases_casesByDay
    │   │   │       ├── COVID-19_cases_countRollingMean
    │   │   │       ├── COVID-19_cases_lineagePercentByWeek
    │   │   │       ├── COVID-19_cases_rateRollingMean
    │   │   │       ├── COVID-19_deaths_ONSByDay
    │   │   │       ├── COVID-19_deaths_ONSRegByWeek
    │   │   │       ├── COVID-19_deaths_ONSRollingMean
    │   │   │       ├── COVID-19_healthcare_admissionByDay
    │   │   │       ├── COVID-19_healthcare_admissionRollingMean
    │   │   │       ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │       ├── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │       ├── COVID-19_testing_PCRcountByDay
    │   │   │       ├── COVID-19_testing_positivity7DayRolling
    │   │   │       ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │       └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   ├── NHS Region
    │   │   │   ├── East of England
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── London
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Midlands
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North East and Yorkshire
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North West
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South East
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   └── South West
    │   │   │       ├── COVID-19_healthcare_admissionByDay
    │   │   │       ├── COVID-19_healthcare_admissionRollingMean
    │   │   │       ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │       └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   ├── NHS Trust
    │   │   │   ├── Airedale NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Alder Hey Children's NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Ashford and St Peter's Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Avon and Wiltshire Mental Health Partnership NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Barking, Havering and Redbridge University Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Barnet, Enfield and Haringey Mental Health NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Barnsley Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Barts Health NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bedfordshire Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Berkshire Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Birmingham and Solihull Mental Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Birmingham Community Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Birmingham Women's and Children's NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Black Country Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Blackpool Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bolton NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bostall House
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bradford District Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bradford Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bridgewater Community Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Bromley Healthcare
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Broomhill
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Buckinghamshire Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Calderdale and Huddersfield NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cambridgeshire and Peterborough NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cambridgeshire Community Services NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cambridge University Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Care Plus Group
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cedar Vale Hospital
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Central and North West London NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Central London Community Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Chelsea and Westminster Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cheshire and Wirral Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Chesterfield Royal Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cheswold Park Hospital
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cheswold Park Hospital (Doncaster)
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── City Health Care Partnership C.I.C
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cornwall Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Countess of Chester Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── County Durham and Darlington NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Coventry and Warwickshire Partnership NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Croydon Health Services NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Csh Surrey
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cumbria, Northumberland, Tyne and Wear NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Acer Clinic
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Alders Clinic
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Cedars
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Churchill
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Elms
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Heathers
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Beckton
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Bierley
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Bury
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Clifton
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Colchester
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Derby
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Ealing
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Godden Green
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Harrogate
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Harrow
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Kewstoke
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Maidstone
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Sheffield
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Stevenage
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Taunton
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Woking
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Hospital Wyke
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Lodge Brighouse
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Lodge Kenton
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Lodge Lewisham
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Manor
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Oaks
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Raglan House
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Sherwood House
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet St. Augustine's
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Storthfield House
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Views
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Cygnet Wing Blackheath
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Dartford and Gravesham NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Derbyshire Community Health Services NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Derbyshire Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Devon Partnership NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Doncaster and Bassetlaw Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Dorset County Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Dorset Healthcare University NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East and North Hertfordshire NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East Cheshire NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East Coast Community Healthcare C.I.C
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East Kent Hospitals University NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East Lancashire Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East London NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East Suffolk and North Essex NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── East Sussex Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Ellern Mede Ridgeway
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Epsom and St Helier University Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Essex Partnership University NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── First Community Health and Care C.I.C
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Frimley Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Gateshead Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── George Eliot Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Gloucestershire Health and Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Gloucestershire Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Greater Manchester Mental Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Great Ormond Street Hospital For Children NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Great Western Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Guy's and St Thomas' NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Hampshire Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Harrogate and District NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Herefordshire and Worcestershire Health and Care NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Hertfordshire Community NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Hertfordshire Partnership University NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Homerton Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   ├── Homerton University Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Hounslow and Richmond Community Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Hull University Teaching Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Humber Teaching NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Imperial College Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Isle of Wight NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── James Paget University Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Kent and Medway NHS and Social Care Partnership Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Kent Community Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Kettering General Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── King's College Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Kingston Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Lancashire & South Cumbria NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Lancashire Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Leeds and York Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Leeds Community Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Leeds Teaching Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Leicestershire Partnership NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Lewisham and Greenwich NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Lincolnshire Community Health Services NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Lincolnshire Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Liverpool Heart and Chest Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Liverpool University Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Liverpool Women's NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Livewell Southwest
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Locala Community Partnerships C.I.C
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── London North West University Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Maidstone and Tunbridge Wells NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Manchester University NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Medway Community Healthcare
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Medway NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Mersey and West Lancashire Teaching Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Mersey Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Mid and South Essex NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Mid Cheshire Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Midlands Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Mid Yorkshire Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Mid Yorkshire Teaching NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Milton Keynes University Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Moorfields Eye Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Navigo Health and Social Care C.I.C
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Newbridge Care Systems
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Norfolk and Norwich University Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Norfolk and Suffolk NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Norfolk Community Health and Care NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Northampton General Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Northamptonshire Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North Bristol NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North Cumbria Integrated Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North East London NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Northern Care Alliance NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   ├── Northern Care Alliance NHS Ft
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Northern Lincolnshire and Goole NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North Middlesex University Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North Staffordshire Combined Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North Tees and Hartlepool NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Northumbria Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── North West Anglia NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Nottingham Citycare Partnership
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Nottinghamshire Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Nottingham University Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Oxford Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Oxford University Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Oxleas NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Pennine Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Portsmouth Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Portsmouth Hospitals University National Health Service Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Provide
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Queen Victoria Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Rotherham Doncaster and South Humber NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Berkshire NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Cornwall Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Devon and Exeter NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Devon University Healthcare NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Free London NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal National Orthopaedic Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Papworth Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal Surrey County Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Royal United Hospitals Bath NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Salford Royal NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Salisbury NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sandwell and West Birmingham Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sheffield Children's NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sheffield Health & Social Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sheffield Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sherwood Forest Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Shrewsbury and Telford Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Shropshire Community Health NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sirona Care & Health
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Solent NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Somerset NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Southern Health NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South London and Maudsley NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South Tees Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South Tyneside and Sunderland NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South Warwickshire NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South Warwickshire University NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South West London and St George's Mental Health NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── South West Yorkshire Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── St Andrew's Healthcare - Birmingham
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── St Andrew's Healthcare - Essex
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── St Andrew's Healthcare - Northampton
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── St Andrew's Healthcare Nottinghamshire
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── St George's University Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── St Magnus Hospital
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Stockport NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Surrey and Borders Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Surrey and Sussex Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sussex Community NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Sussex Partnership NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Tameside and Glossop Integrated Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Tavistock and Portman NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Tees, Esk and Wear Valleys NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Christie NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Clatterbridge Cancer Centre NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Dallingtons
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Dudley Group NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Hillingdon Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Huntercombe Hospital - Maidenhead
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Huntercombe Hospital - Stafford
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Newcastle Upon Tyne Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Princess Alexandra Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Queen Elizabeth Hospital, King's Lynn, NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Robert Jones and Agnes Hunt Orthopaedic Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Rotherham NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Royal Marsden NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Royal Orthopaedic Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Royal Wolverhampton NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Shrewsbury and Telford Hospital NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── The Walton Centre NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Torbay and South Devon NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── United Lincolnshire Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University College London Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Birmingham NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Bristol and Weston NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Coventry and Warwickshire NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Dorset NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals of Derby and Burton NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals of Leicester NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals of Morecambe Bay NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals of North Midlands NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospital Southampton NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Plymouth NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Sussex NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── University Hospitals Sussex NHS Foundation Trust 
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Walsall Healthcare NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Warrington and Halton Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Wast Hills Hospital
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── West Hertfordshire Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── West Hertfordshire Teaching Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── West London NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── West Suffolk NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Whittington Health NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Wiltshire Health & Care
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Wirral Community Health and Care NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Wirral University Teaching Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Worcestershire Acute Hospitals NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Worcestershire Health and Care NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Wrightington, Wigan and Leigh NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── Wye Valley NHS Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── York and Scarborough Teaching Hospitals NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   ├── York Teaching Hospital NHS Foundation Trust
    │   │   │   │   ├── COVID-19_healthcare_admissionByDay
    │   │   │   │   ├── COVID-19_healthcare_admissionRollingMean
    │   │   │   │   ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │   │   └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   │   └── Your Healthcare
    │   │   │       ├── COVID-19_healthcare_admissionByDay
    │   │   │       ├── COVID-19_healthcare_admissionRollingMean
    │   │   │       ├── COVID-19_healthcare_occupiedBedsByDay
    │   │   │       └── COVID-19_healthcare_occupiedBedsRollingMean
    │   │   ├── UKHSA Region
    │   │   │   ├── East Midlands
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── East of England
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── London
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North East
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── North West
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South East
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── South West
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   ├── West Midlands
    │   │   │   │   ├── COVID-19_cases_casesByDay
    │   │   │   │   ├── COVID-19_cases_countRollingMean
    │   │   │   │   ├── COVID-19_cases_rateRollingMean
    │   │   │   │   ├── COVID-19_testing_PCRcountByDay
    │   │   │   │   ├── COVID-19_testing_positivity7DayRolling
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │   │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │   │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   │   └── Yorkshire and Humber
    │   │   │       ├── COVID-19_cases_casesByDay
    │   │   │       ├── COVID-19_cases_countRollingMean
    │   │   │       ├── COVID-19_cases_rateRollingMean
    │   │   │       ├── COVID-19_testing_PCRcountByDay
    │   │   │       ├── COVID-19_testing_positivity7DayRolling
    │   │   │       ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │   │       ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │   │       ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │   │       └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │   └── Upper Tier Local Authority
    │   │       ├── Barking and Dagenham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Barnet
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Barnsley
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bath and North East Somerset
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bedford
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bexley
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Birmingham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Blackburn with Darwen
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Blackpool
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bolton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bournemouth, Christchurch and Poole
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bracknell Forest
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bradford
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Brent
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Brighton and Hove
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bristol, City of
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bromley
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Buckinghamshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Bury
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Calderdale
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Cambridgeshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Camden
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Central Bedfordshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Cheshire East
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Cheshire West and Chester
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Cornwall and Isles of Scilly
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── County Durham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Coventry
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Croydon
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Cumbria
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Darlington
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Derby
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Derbyshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Devon
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Doncaster
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Dorset
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Dudley
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Ealing
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── East Riding of Yorkshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── East Sussex
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Enfield
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Essex
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Gateshead
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Gloucestershire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Greenwich
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hackney and City of London
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Halton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hammersmith and Fulham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hampshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Haringey
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Harrow
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hartlepool
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Havering
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Herefordshire, County of
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hertfordshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hillingdon
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Hounslow
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Isle of Wight
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Islington
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Kensington and Chelsea
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Kent
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Kingston upon Hull, City of
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Kingston upon Thames
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Kirklees
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Knowsley
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Lambeth
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Lancashire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Leeds
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Leicester
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Leicestershire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Lewisham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Lincolnshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Liverpool
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Luton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Manchester
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Medway
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Merton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Middlesbrough
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Milton Keynes
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Newcastle upon Tyne
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Newham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Norfolk
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── North East Lincolnshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── North Lincolnshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── North Northamptonshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── North Somerset
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── North Tyneside
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Northumberland
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── North Yorkshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Nottingham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Nottinghamshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Oldham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Oxfordshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Peterborough
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Plymouth
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Portsmouth
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Reading
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Redbridge
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Redcar and Cleveland
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Richmond upon Thames
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Rochdale
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Rotherham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Rutland
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Salford
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Sandwell
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Sefton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Sheffield
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Shropshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Slough
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Solihull
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Somerset
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Southampton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Southend-on-Sea
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── South Gloucestershire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── South Tyneside
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Southwark
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Staffordshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── St. Helens
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Stockport
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Stockton-on-Tees
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Stoke-on-Trent
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Suffolk
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Sunderland
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Surrey
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Sutton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Swindon
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Tameside
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Telford and Wrekin
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Thurrock
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Torbay
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Tower Hamlets
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Trafford
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wakefield
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Walsall
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Waltham Forest
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wandsworth
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Warrington
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Warwickshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── West Berkshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Westminster
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── West Northamptonshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── West Sussex
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wigan
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wiltshire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Windsor and Maidenhead
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wirral
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wokingham
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Wolverhampton
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       ├── Worcestershire
    │   │       │   ├── COVID-19_cases_casesByDay
    │   │       │   ├── COVID-19_cases_countRollingMean
    │   │       │   ├── COVID-19_cases_rateRollingMean
    │   │       │   ├── COVID-19_testing_PCRcountByDay
    │   │       │   ├── COVID-19_testing_positivity7DayRolling
    │   │       │   ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │       │   ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │       │   ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │       │   └── COVID-19_vaccinations_spring24_uptakeByDay
    │   │       └── York
    │   │           ├── COVID-19_cases_casesByDay
    │   │           ├── COVID-19_cases_countRollingMean
    │   │           ├── COVID-19_cases_rateRollingMean
    │   │           ├── COVID-19_testing_PCRcountByDay
    │   │           ├── COVID-19_testing_positivity7DayRolling
    │   │           ├── COVID-19_vaccinations_autumn22_dosesByDay
    │   │           ├── COVID-19_vaccinations_autumn22_uptakeByDay
    │   │           ├── COVID-19_vaccinations_autumn23_dosesByDay
    │   │           ├── COVID-19_vaccinations_autumn23_uptakeByDay
    │   │           ├── COVID-19_vaccinations_autumn24_dosesByDay
    │   │           ├── COVID-19_vaccinations_autumn24_uptakeByDay
    │   │           ├── COVID-19_vaccinations_spring23_dosesByDay
    │   │           ├── COVID-19_vaccinations_spring23_uptakeByDay
    │   │           ├── COVID-19_vaccinations_spring24_dosesByDay
    │   │           └── COVID-19_vaccinations_spring24_uptakeByDay
    │   ├── hMPV
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── hMPV_testing_positivityByWeek
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── East of England
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── London
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── North East
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── North West
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── South East
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── South West
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       ├── West Midlands
    │   │       │   └── hMPV_testing_positivityByWeek
    │   │       └── Yorkshire and Humber
    │   │           └── hMPV_testing_positivityByWeek
    │   ├── Influenza
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │       ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │       └── influenza_testing_positivityByWeek
    │   │   ├── UKHSA Region
    │   │   │   ├── East Midlands
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── East of England
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── London
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── North East
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── North West
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── South East
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── South West
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   ├── West Midlands
    │   │   │   │   ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │   │   ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │   │   └── influenza_testing_positivityByWeek
    │   │   │   └── Yorkshire and Humber
    │   │   │       ├── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │   │       ├── influenza_healthcare_ICUHDUadmissionRateByWeek
    │   │   │       └── influenza_testing_positivityByWeek
    │   │   └── UKHSA Super-Region
    │   │       ├── London
    │   │       │   └── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │       ├── Midlands and East of England
    │   │       │   └── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │       ├── North of England
    │   │       │   └── influenza_healthcare_hospitalAdmissionRateByWeek
    │   │       └── South of England
    │   │           └── influenza_healthcare_hospitalAdmissionRateByWeek
    │   ├── Parainfluenza
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── parainfluenza_testing_positivityByWeek
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── East of England
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── London
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── North East
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── North West
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── South East
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── South West
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       ├── West Midlands
    │   │       │   └── parainfluenza_testing_positivityByWeek
    │   │       └── Yorkshire and Humber
    │   │           └── parainfluenza_testing_positivityByWeek
    │   ├── Rhinovirus
    │   │   ├── Nation
    │   │   │   └── England
    │   │   │       └── rhinovirus_testing_positivityByWeek
    │   │   └── UKHSA Region
    │   │       ├── East Midlands
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── East of England
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── London
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── North East
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── North West
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── South East
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── South West
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       ├── West Midlands
    │   │       │   └── rhinovirus_testing_positivityByWeek
    │   │       └── Yorkshire and Humber
    │   │           └── rhinovirus_testing_positivityByWeek
    │   └── RSV
    │       ├── Nation
    │       │   └── England
    │       │       ├── RSV_healthcare_admissionRateByWeek
    │       │       └── RSV_testing_positivityByWeek
    │       └── UKHSA Region
    │           ├── East Midlands
    │           │   └── RSV_testing_positivityByWeek
    │           ├── East of England
    │           │   └── RSV_testing_positivityByWeek
    │           ├── London
    │           │   └── RSV_testing_positivityByWeek
    │           ├── North East
    │           │   └── RSV_testing_positivityByWeek
    │           ├── North West
    │           │   └── RSV_testing_positivityByWeek
    │           ├── South East
    │           │   └── RSV_testing_positivityByWeek
    │           ├── South West
    │           │   └── RSV_testing_positivityByWeek
    │           ├── West Midlands
    │           │   └── RSV_testing_positivityByWeek
    │           └── Yorkshire and Humber
    │               └── RSV_testing_positivityByWeek
    └── vaccine_preventable
        └── Measles
            ├── Nation
            │   └── England
            │       ├── measles_cases_casesByOnsetWeek
            │       └── measles_cases_casesByOnsetWeekExcludingReportingLag
            └── UKHSA Region
                ├── East Midlands
                │   └── measles_cases_casesByOnsetWeek
                ├── East of England
                │   └── measles_cases_casesByOnsetWeek
                ├── London
                │   └── measles_cases_casesByOnsetWeek
                ├── North East
                │   └── measles_cases_casesByOnsetWeek
                ├── North West
                │   └── measles_cases_casesByOnsetWeek
                ├── South East
                │   └── measles_cases_casesByOnsetWeek
                ├── South West
                │   └── measles_cases_casesByOnsetWeek
                ├── West Midlands
                │   └── measles_cases_casesByOnsetWeek
                └── Yorkshire and Humber
                    └── measles_cases_casesByOnsetWeek

```